# Importação das bibliotecas e criação de funções auxiliares


In [1]:
import numpy as np # Biblioteca para manipulação de matrizes
import pandas as pd # Biblioteca para manipulação de datasets
import statistics as stats # Biblioteca para aplicação de funções estatísticas
from scipy import stats as stats2 # Biblioteca para aplicação de funções estatísticas
import cv2 # Biblioteca para visão computacional
import os # Biblioteca do Sistema Operacional
import matplotlib.pyplot as plt # Biblioteca pra visualização de dados
import seaborn as sns # Biblioteca pra visualização de dados
from sklearn.metrics import auc # Biblioteca para cálculo de área sobre a curva (utilizada na função get_features)
% matplotlib inline

import time # Bibliteca para cálculo de tempo
start_time = time.time()

In [2]:
def get_int(val):

  ### Função de arredondamento para Theta ###

  ''' Esta função foi criada, como mostra na etapa de introdução, para realizar um ajuste nos pontos de contorno de cada imagem,
      multiplicando eles por 10⁶ para realizar um arredondamento e ordenalos de maneira correta, fazendo com que o DataFrame que
      que utilizará estes valores para ordenar os valores de Theta'''

  Value = val*10**6
  Inteiro = round(Value)
  return Inteiro

In [3]:
img_folder_path  = '/content/drive/My Drive/Colab Notebooks/TCC/PH2/Dataset/PH2_Separado/IMD/'
mask_folder_path = '/content/drive/My Drive/Colab Notebooks/TCC/PH2/Dataset/PH2_Separado/lesion/'
img_extension = '.bmp'
mask_extension = '_lesion.bmp'

def get_features(img):

  ### Função responsável por retirar as características da imagem ###

  ''' Função responsável por retirar todas as informações pertinentes sobre a imagem, analisando desde a coloração na região de
      importância, até características geométricas como raio, assimetria e borda. Cada etapa do código é explicada a seguir.

  '''

  ## Carregando a imagem e sua máscara e realizando as transformações de cores ##
  path     = img_folder_path + img
  img_BGR  = cv2.imread(path + img_extension)
  img_RGB  = cv2.cvtColor(img_BGR,cv2.COLOR_BGR2RGB)
  img_gray = cv2.cvtColor(img_BGR,cv2.COLOR_BGR2GRAY)
  img_HSL  = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HLS)
  img_HSV  = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
  img_LAB  = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2Lab)
  img_LUV  = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2Luv)
  mask_bin = cv2.imread(mask_folder_path + img+ mask_extension,0)

  ## Configurando especificações de altura e largura da imagem  ##
  shape = img_BGR.shape
  alt  = shape[0]
  larg = shape[1]
  h = alt
  w = larg

  ## Criação de imagens auxiliares para criação do contorno e da menor elipse  ##
  black1 = np.zeros((alt,larg,3),np.uint8)
  black2 = black1.copy()


  ## Binarização da imagem e cálculo dos momentos da imagem ##
  ret, th1 = cv2.threshold(mask_bin,127,255,0)
  contours,hierarchy = cv2.findContours(th1, 1, 2)
  cnt = contours[0]
  M = cv2.moments(cnt)
  cx = int(M['m10']/M['m00'])
  cy = int(M['m01']/M['m00'])

  
  ## Criação do contorno e da menor elipse utilizando os valores do contorno ##
  contorno = cv2.drawContours(black1, contours, -1, (0,255,255), 1)
  ellipse_stats = cv2.fitEllipse(cnt)
  ellipse = cv2.ellipse(black2,ellipse_stats,(255,255,255),-1)
  ellipse = cv2.cvtColor(ellipse,cv2.COLOR_BGR2GRAY)
  ret, th2 = cv2.threshold(ellipse,127,255,0)
  contours, hierarchy = cv2.findContours(th2, 1, 2)
  ellipse = cv2.drawContours(black2, contours, -1, (0,255,255), 0)




  ### Preparação de variáveis de armazenamento para análise ###

  # Variáveis para COR #
  R = []
  B = []
  G = []
  Gray = []

  R_adap = []
  B_adap = []
  G_adap = []
  Gray_adap = []

  HSL_H,HSL_S,HSL_L = [],[],[]
  HSV_H,HSV_S,HSV_V = [],[],[]
  LAB_L,LAB_A,LAB_B = [],[],[]
  LUV_L,LUV_U,LUV_V = [],[],[] 

  # Variáveis para ASSIMETRIA #
  sumq1, sumq2, sumq3, sumq4 = 0,0,0,0

  # Variáveis para armazenar os PONTOS pertencentes as bordas do contorno e da elipse #
  PQ1xCont, PQ1yCont = [], []
  PQ2xCont, PQ2yCont = [], []
  PQ3xCont, PQ3yCont = [], []
  PQ4xCont, PQ4yCont = [], []
  PQ1xElip, PQ1yElip = [], []
  PQ2xElip, PQ2yElip = [], []
  PQ3xElip, PQ3yElip = [], []
  PQ4xElip, PQ4yElip = [], []
  

  ## Varredura de toda imagem, buscando os valores pertinentes ##
  for i in range(0,alt):
    for j in range(0,larg):
      if th1[i,j] > 0:
        # Sistema RGB e Cinza
        R.append(img_RGB[i,j,0])
        G.append(img_RGB[i,j,1])
        B.append(img_RGB[i,j,2])
        Gray.append(img_gray[i,j])

        # Sistema RGB_adap
        if img_RGB[i,j,0] == 0:
          R_adap.append(0.01)
        else:
          R_adap.append(img_RGB[i,j,0])
        
        if img_RGB[i,j,1] == 0:
          G_adap.append(0.01)
        else:
          G_adap.append(img_RGB[i,j,1])

        if img_RGB[i,j,2] == 0:
          B_adap.append(0.01)
        else:
          B_adap.append(img_RGB[i,j,2])

        if img_gray[i,j] == 0:
          Gray_adap.append(0.01)
        else:
          Gray_adap.append(img_gray[i,j])


        # Sistema HSL
        if img_HSL[i,j,0] == 0:
          HSL_H.append(0.01)
        else:
          HSL_H.append(img_HSL[i,j,0])

        if img_HSL[i,j,1] == 0:
          HSL_S.append(0.01)
        else:
          HSL_S.append(img_HSL[i,j,1])

        if img_HSL[i,j,2] == 0:
          HSL_L.append(0.01)
        else:
          HSL_L.append(img_HSL[i,j,2])


        # Sistema HSV
        if img_HSV[i,j,0] == 0:
          HSV_H.append(0.01)
        else:
          HSV_H.append(img_HSV[i,j,0])

        if img_HSV[i,j,1] == 0:
          HSV_S.append(0.01)
        else:
          HSV_S.append(img_HSV[i,j,1])

        if img_HSV[i,j,2] == 0:
          HSV_V.append(0.01)
        else:
          HSV_V.append(img_HSV[i,j,2])


        # Sistema LAB
        if img_LAB[i,j,0] == 0:
          LAB_L.append(0.01)
        else:
          LAB_L.append(img_LAB[i,j,0])

        if img_LAB[i,j,1] == 0:
          LAB_A.append(0.01)
        else:
          LAB_A.append(img_LAB[i,j,1])

        if img_LAB[i,j,2] == 0:
          LAB_B.append(0.01)
        else:
          LAB_B.append(img_LAB[i,j,2])



        # Sistema LAB
        if img_LUV[i,j,0] == 0:
          LUV_L.append(0.01)
        else:
          LUV_L.append(img_LUV[i,j,0])

        if img_LUV[i,j,1] == 0:
          LUV_U.append(0.01)
        else:
          LUV_U.append(img_LUV[i,j,1])

        if img_LUV[i,j,2] == 0:
          LUV_V.append(0.01)
        else:
          LUV_V.append(img_LUV[i,j,2])


        if j < cx:
          if i < cy:
            # Primeiro Quadrante
            sumq1 += 1
          else:
            # Terceiro Quadrante
            sumq3 += 1
        else:
          if i < cy:
            # Segundo Quadrante
            sumq2 += 1 
          else: 
            # Quarto Quadrante
            sumq4 += 1

      if contorno[i,j,1] > 0:
        if j < cx:
          if i < cy:
            # Segundo Quadrante
            PQ2xCont.append(j)
            PQ2yCont.append(i)
          
          else:
             # Terceiro Quadrante
             PQ3xCont.append(j)
             PQ3yCont.append(i)

        else:
          if i < cy:
             # Primeiro Quadrante
             PQ1xCont.append(j)
             PQ1yCont.append(i)

          else:
            # Quarto Quadrante
            PQ4xCont.append(j)
            PQ4yCont.append(i)
            
      if ellipse[i,j,1] > 0:
        if j < cx:
          if i < cy:
            # Segundo Quadrante
            PQ2xElip.append(j)
            PQ2yElip.append(i)
          
          else:
             # Terceiro Quadrante
             PQ3xElip.append(j)
             PQ3yElip.append(i)

        else:
          if i < cy:
             # Primeiro Quadrante
             PQ1xElip.append(j)
             PQ1yElip.append(i)

          else:
            # Quarto Quadrante
            PQ4xElip.append(j)
            PQ4yElip.append(i)  


  ## Conversão de lista em vetor para realizar os cálculos ##
  R = pd.Series(R)
  G = pd.Series(G)
  B = pd.Series(B)
  Gray = pd.Series(Gray)

  R_adap = pd.Series(R_adap)
  B_adap = pd.Series(B_adap)
  G_adap = pd.Series(G_adap)
  Gray_adap = pd.Series(Gray_adap)

  HSL_H = pd.Series(HSL_H)
  HSL_S = pd.Series(HSL_S)
  HSL_L = pd.Series(HSL_L)


  HSV_H = pd.Series(HSV_H)
  HSV_S = pd.Series(HSV_S)
  HSV_V = pd.Series(HSV_V)


  LAB_L = pd.Series(LAB_L)
  LAB_A = pd.Series(LAB_A)
  LAB_B = pd.Series(LAB_B)

  
  LUV_L = pd.Series(LUV_L)
  LUV_U = pd.Series(LUV_U)
  LUV_V = pd.Series(LUV_V)

  # Cálculos do sistema RGB e Cinza
  Rmed = R.mean()
  Rvar = R.var()
  Rstd = R.std()
  Rmax = R.max()
  Rmin = R.min()
  Rmdn = R.median()
  RQ25 = R.quantile(q=0.25)
  RQ75 = R.quantile(q=0.75)
  Rmod = R.mode()[0]
  Ramp = Rmax - Rmin
  Rsta = R.mad()
  Rgeo = stats2.gmean(R)
  Rhar = stats2.hmean(R)
  Rmode_len = len(R.mode())

  Gmed = G.mean()
  Gvar = G.var()
  Gstd = G.std()
  Gmax = G.max()
  Gmin = G.min()
  Gmdn = G.median()
  GQ25 = G.quantile(q=0.25)
  GQ75 = G.quantile(q=0.75)
  Gmod = G.mode()[0]
  Gamp = Gmax - Gmin
  Gsta = G.mad()
  Ggeo = stats2.gmean(G)
  Ghar = stats2.hmean(G)
  Gmode_len = len(G.mode())


  Bmed = B.mean()
  Bvar = B.var()
  Bstd = B.std()
  Bmax = B.max()
  Bmin = B.min()
  Bmdn = B.median()
  BQ25 = B.quantile(q=0.25)
  BQ75 = B.quantile(q=0.75)
  Bmod = B.mode()[0]
  Bamp = Bmax - Bmin
  Bsta = B.mad()
  Bgeo = stats2.gmean(B)
  Bhar = stats2.hmean(B)
  Bmode_len = len(B.mode())

  Graymed = Gray.mean()
  Grayvar = Gray.var()
  Graystd = Gray.std()
  Graymax = Gray.max()
  Graymin = Gray.min()
  Graymdn = Gray.median()
  GrayQ25 = Gray.quantile(q=0.25)
  GrayQ75 = Gray.quantile(q=0.75)
  Graymod = Gray.mode()[0]
  Grayamp = Graymax - Graymin
  Graysta = Gray.mad()
  Graygeo = stats2.gmean(Gray)
  Grayhar = stats2.hmean(Gray)
  Graymode_len = len(Gray.mode())

  # Cálculos do sistema RGB_adap
  Rmed_adap = R_adap.mean()
  Rvar_adap = R_adap.var()
  Rstd_adap = R_adap.std()
  Rmax_adap = R_adap.max()
  Rmin_adap = R_adap.min()
  Rmdn_adap = R_adap.median()
  RQ25_adap = R_adap.quantile(q=0.25)
  RQ75_adap = R_adap.quantile(q=0.75)
  Rmod_adap = R_adap.mode()[0]
  Ramp_adap = Rmax_adap - Rmin_adap
  Rsta_adap = R_adap.mad()
  Rgeo_adap = stats2.gmean(R_adap)
  Rhar_adap = stats2.hmean(R_adap)
  Rmode_adap_len = len(R_adap.mode())

  Gmed_adap = G_adap.mean()
  Gvar_adap = G_adap.var()
  Gstd_adap = G_adap.std()
  Gmax_adap = G_adap.max()
  Gmin_adap = G_adap.min()
  Gmdn_adap = G_adap.median()
  GQ25_adap = G_adap.quantile(q=0.25)
  GQ75_adap = G_adap.quantile(q=0.75)
  Gmod_adap = G_adap.mode()[0]
  Gamp_adap = Gmax_adap - Gmin_adap
  Gsta_adap = G_adap.mad()
  Ggeo_adap = stats2.gmean(G_adap)
  Ghar_adap = stats2.hmean(G_adap)
  Gmode_adap_len = len(G_adap.mode())


  Bmed_adap = B_adap.mean()
  Bvar_adap = B_adap.var()
  Bstd_adap = B_adap.std()
  Bmax_adap = B_adap.max()
  Bmin_adap = B_adap.min()
  Bmdn_adap = B_adap.median()
  BQ25_adap = B_adap.quantile(q=0.25)
  BQ75_adap = B_adap.quantile(q=0.75)
  Bmod_adap = B_adap.mode()[0]
  Bamp_adap = Bmax_adap - Bmin_adap
  Bsta_adap = B_adap.mad()
  Bgeo_adap = stats2.gmean(B_adap)
  Bhar_adap = stats2.hmean(B_adap)
  Bmode_adap_len = len(B_adap.mode())

  Graymed_adap = Gray_adap.mean()
  Grayvar_adap = Gray_adap.var()
  Graystd_adap = Gray_adap.std()
  Graymax_adap = Gray_adap.max()
  Graymin_adap = Gray_adap.min()
  Graymdn_adap = Gray_adap.median()
  GrayQ25_adap = Gray_adap.quantile(q=0.25)
  GrayQ75_adap = Gray_adap.quantile(q=0.75)
  Graymod_adap = Gray_adap.mode()[0]
  Grayamp_adap = Graymax_adap - Graymin_adap
  Graysta_adap = Gray_adap.mad()
  Graygeo_adap = stats2.gmean(Gray_adap)
  Grayhar_adap = stats2.hmean(Gray_adap)
  Graymode_adap_len = len(Gray_adap.mode())

  # Cálculos do sistema HSL
  HSL_Hmed = HSL_H.mean()
  HSL_Hvar = HSL_H.var()
  HSL_Hstd = HSL_H.std()
  HSL_Hmax = HSL_H.max()
  HSL_Hmin = HSL_H.min()
  HSL_Hmdn = HSL_H.median()
  HSL_HQ25 = HSL_H.quantile(q=0.25)
  HSL_HQ75 = HSL_H.quantile(q=0.75)
  HSL_Hmod = HSL_H.mode()[0]
  HSL_Hamp = HSL_Hmax - HSL_Hmin
  HSL_Hsta = HSL_H.mad()
  HSL_Hgeo = stats2.gmean(HSL_H)
  HSL_Hhar = stats2.hmean(HSL_H)
  HSL_Hmode_len = len(HSL_H.mode())

  HSL_Smed = HSL_S.mean()
  HSL_Svar = HSL_S.var()
  HSL_Sstd = HSL_S.std()
  HSL_Smax = HSL_S.max()
  HSL_Smin = HSL_S.min()
  HSL_Smdn = HSL_S.median()
  HSL_SQ25 = HSL_S.quantile(q=0.25)
  HSL_SQ75 = HSL_S.quantile(q=0.75)
  HSL_Smod = HSL_S.mode()[0]
  HSL_Samp = HSL_Smax - HSL_Smin
  HSL_Ssta = HSL_S.mad()
  HSL_Sgeo = stats2.gmean(HSL_S)
  HSL_Shar = stats2.hmean(HSL_S)
  HSL_Smode_len = len(HSL_S.mode())


  HSL_Lmed = HSL_L.mean()
  HSL_Lvar = HSL_L.var()
  HSL_Lstd = HSL_L.std()
  HSL_Lmax = HSL_L.max()
  HSL_Lmin = HSL_L.min()
  HSL_Lmdn = HSL_L.median()
  HSL_LQ25 = HSL_L.quantile(q=0.25)
  HSL_LQ75 = HSL_L.quantile(q=0.75)
  HSL_Lmod = HSL_L.mode()[0]
  HSL_Lamp = HSL_Lmax - HSL_Lmin
  HSL_Lsta = HSL_L.mad()
  HSL_Lgeo = stats2.gmean(HSL_L)
  HSL_Lhar = stats2.hmean(HSL_L)
  HSL_Lmode_len = len(HSL_L.mode())

  # Cálculos do sistema HSV
  HSV_Hmed = HSV_H.mean()
  HSV_Hvar = HSV_H.var()
  HSV_Hstd = HSV_H.std()
  HSV_Hmax = HSV_H.max()
  HSV_Hmin = HSV_H.min()
  HSV_Hmdn = HSV_H.median()
  HSV_HQ25 = HSV_H.quantile(q=0.25)
  HSV_HQ75 = HSV_H.quantile(q=0.75)
  HSV_Hmod = HSV_H.mode()[0]
  HSV_Hamp = HSV_Hmax - HSV_Hmin
  HSV_Hsta = HSV_H.mad()
  HSV_Hgeo = stats2.gmean(HSV_H)
  HSV_Hhar = stats2.hmean(HSV_H)
  HSV_Hmode_len = len(HSV_H.mode())

  HSV_Smed = HSV_S.mean()
  HSV_Svar = HSV_S.var()
  HSV_Sstd = HSV_S.std()
  HSV_Smax = HSV_S.max()
  HSV_Smin = HSV_S.min()
  HSV_Smdn = HSV_S.median()
  HSV_SQ25 = HSV_S.quantile(q=0.25)
  HSV_SQ75 = HSV_S.quantile(q=0.75)
  HSV_Smod = HSV_S.mode()[0]
  HSV_Samp = HSV_Smax - HSV_Smin
  HSV_Ssta = HSV_S.mad()
  HSV_Sgeo = stats2.gmean(HSV_S)
  HSV_Shar = stats2.hmean(HSV_S)
  HSV_Smode_len = len(HSV_S.mode())


  HSV_Vmed = HSV_V.mean()
  HSV_Vvar = HSV_V.var()
  HSV_Vstd = HSV_V.std()
  HSV_Vmax = HSV_V.max()
  HSV_Vmin = HSV_V.min()
  HSV_Vmdn = HSV_V.median()
  HSV_VQ25 = HSV_V.quantile(q=0.25)
  HSV_VQ75 = HSV_V.quantile(q=0.75)
  HSV_Vmod = HSV_V.mode()[0]
  HSV_Vamp = HSV_Vmax - HSV_Vmin
  HSV_Vsta = HSV_V.mad()
  HSV_Vgeo = stats2.gmean(HSV_V)
  HSV_Vhar = stats2.hmean(HSV_V)
  HSV_Vmode_len = len(HSV_V.mode())

  # Cálculos do sistema LAB
  LAB_Lmed = LAB_L.mean()
  LAB_Lvar = LAB_L.var()
  LAB_Lstd = LAB_L.std()
  LAB_Lmax = LAB_L.max()
  LAB_Lmin = LAB_L.min()
  LAB_Lmdn = LAB_L.median()
  LAB_LQ25 = LAB_L.quantile(q=0.25)
  LAB_LQ75 = LAB_L.quantile(q=0.75)
  LAB_Lmod = LAB_L.mode()[0]
  LAB_Lamp = LAB_Lmax - LAB_Lmin
  LAB_Lsta = LAB_L.mad()
  LAB_Lgeo = stats2.gmean(LAB_L)
  LAB_Lhar = stats2.hmean(LAB_L)
  LAB_Lmode_len = len(LAB_L.mode())

  LAB_Amed = LAB_A.mean()
  LAB_Avar = LAB_A.var()
  LAB_Astd = LAB_A.std()
  LAB_Amax = LAB_A.max()
  LAB_Amin = LAB_A.min()
  LAB_Amdn = LAB_A.median()
  LAB_AQ25 = LAB_A.quantile(q=0.25)
  LAB_AQ75 = LAB_A.quantile(q=0.75)
  LAB_Amod = LAB_A.mode()[0]
  LAB_Aamp = LAB_Amax - LAB_Amin
  LAB_Asta = LAB_A.mad()
  LAB_Ageo = stats2.gmean(LAB_A)
  LAB_Ahar = stats2.hmean(LAB_A)
  LAB_Amode_len = len(LAB_A.mode())


  LAB_Bmed =  LAB_B.mean()
  LAB_Bvar =  LAB_B.var()
  LAB_Bstd =  LAB_B.std()
  LAB_Bmax =  LAB_B.max()
  LAB_Bmin =  LAB_B.min()
  LAB_Bmdn =  LAB_B.median()
  LAB_BQ25 =  LAB_B.quantile(q=0.25)
  LAB_BQ75 =  LAB_B.quantile(q=0.75)
  LAB_Bmod =  LAB_B.mode()[0]
  LAB_Bamp =  LAB_Bmax -  LAB_Bmin
  LAB_Bsta =  LAB_B.mad()
  LAB_Bgeo = stats2.gmean(LAB_B)
  LAB_Bhar = stats2.hmean(LAB_B)
  LAB_Bmode_len = len(LAB_B.mode())

  # Cálculos do sistema LUV
  LUV_Lmed = LUV_L.mean()
  LUV_Lvar = LUV_L.var()
  LUV_Lstd = LUV_L.std()
  LUV_Lmax = LUV_L.max()
  LUV_Lmin = LUV_L.min()
  LUV_Lmdn = LUV_L.median()
  LUV_LQ25 = LUV_L.quantile(q=0.25)
  LUV_LQ75 = LUV_L.quantile(q=0.75)
  LUV_Lmod = LUV_L.mode()[0]
  LUV_Lamp = LUV_Lmax - LUV_Lmin
  LUV_Lsta = LUV_L.mad()
  LUV_Lgeo = stats2.gmean(LUV_L)
  LUV_Lhar = stats2.hmean(LUV_L)
  LUV_Lmode_len = len(LUV_L.mode())

  LUV_Umed = LUV_U.mean()
  LUV_Uvar = LUV_U.var()
  LUV_Ustd = LUV_U.std()
  LUV_Umax = LUV_U.max()
  LUV_Umin = LUV_U.min()
  LUV_Umdn = LUV_U.median()
  LUV_UQ25 = LUV_U.quantile(q=0.25)
  LUV_UQ75 = LUV_U.quantile(q=0.75)
  LUV_Umod = LUV_U.mode()[0]
  LUV_Uamp = LUV_Umax - LUV_Umin
  LUV_Usta = LUV_U.mad()
  LUV_Ugeo = stats2.gmean(LUV_U)
  LUV_Uhar = stats2.hmean(LUV_U)
  LUV_Umode_len = len(LUV_U.mode())

  LUV_Vmed = LUV_V.mean()
  LUV_Vvar = LUV_V.var()
  LUV_Vstd = LUV_V.std()
  LUV_Vmax = LUV_V.max()
  LUV_Vmin = LUV_V.min()
  LUV_Vmdn = LUV_V.median()
  LUV_VQ25 = LUV_V.quantile(q=0.25)
  LUV_VQ75 = LUV_V.quantile(q=0.75)
  LUV_Vmod = LUV_V.mode()[0]
  LUV_Vamp = LUV_Vmax - LUV_Vmin
  LUV_Vsta = LUV_V.mad()
  LUV_Vgeo = stats2.gmean(LUV_V)
  LUV_Vhar = stats2.hmean(LUV_V)
  LUV_Vmode_len = len(LUV_V.mode())



  ## Obtenção do raio, perímetro e área através dos momentos calculados ##
  (x,y),radius = cv2.minEnclosingCircle(cnt)
  raio = int(radius)
  area = cv2.contourArea(cnt)
  perimetro = cv2.arcLength(cnt,True)


  ## Cálculo de assimetria em X e em Y ## ------------
  assymx = np.absolute(sumq1+sumq3 - (sumq2+sumq4)) #|
  assymy = np.absolute(sumq1+sumq2 - (sumq3+sumq4)) #|
                                                   
  ## Conversão de lista para vetor dos pontos percetences a elipse e contorno  ##
  PQ1xCont = np.asarray(PQ1xCont)
  PQ1yCont = np.asarray(PQ1yCont)
  PQ2xCont = np.asarray(PQ2xCont)
  PQ2yCont = np.asarray(PQ2yCont)
  PQ3xCont = np.asarray(PQ3xCont)
  PQ3yCont = np.asarray(PQ3yCont)
  PQ4xCont = np.asarray(PQ4xCont)
  PQ4yCont = np.asarray(PQ4yCont)


  PQ1xElip = np.asarray(PQ1xElip)
  PQ1yElip = np.asarray(PQ1yElip)
  PQ2xElip = np.asarray(PQ2xElip)
  PQ2yElip = np.asarray(PQ2yElip)
  PQ3xElip = np.asarray(PQ3xElip)
  PQ3yElip = np.asarray(PQ3yElip)
  PQ4xElip = np.asarray(PQ4xElip)
  PQ4yElip = np.asarray(PQ4yElip)


  PxCont = np.concatenate([PQ1xCont,PQ2xCont,PQ3xCont,PQ4xCont])
  PyCont = np.concatenate([PQ1yCont,PQ2yCont,PQ3yCont,PQ4yCont])

  PxElip = np.concatenate([PQ1xElip,PQ2xElip,PQ3xElip,PQ4xElip])
  PyElip = np.concatenate([PQ1yElip,PQ2yElip,PQ3yElip,PQ4yElip])

  ## Translação dos pontos para a origem e rotação da imagem ##
  PX_cont = PxCont-cx
  PY_cont = -PyCont+cy

  PX_elip = PxElip-cx
  PY_elip = -PyElip+cy

  ## Cálculo da distância do centro até a borda e de theta ##
  R_cont = np.sqrt(PX_cont**2+PY_cont**2)

  Theta_cont = []

  for i in range(0,len(R_cont)):
    cos = PX_cont[i]/R_cont[i]
    if cos > 1:
      cos = 1
    if cos < -1:
      cos = -1
    else:
      cos = cos
    Theta = np.degrees(np.arccos(cos))
    Theta_cont.append(Theta)

  Theta_cont = np.asarray(Theta_cont)

  R_elip = np.sqrt(PX_elip**2+PY_elip**2)

  Theta_elip = []

  for i in range(0,len(R_elip)):
    cos = PX_elip[i]/R_elip[i]
    if cos > 1:
      cos = 1
    if cos < -1:
      cos = -1
    else:
      cos = cos
    Theta = np.degrees(np.arccos(cos))
    Theta_elip.append(Theta)

  Theta_elip = np.asarray(Theta_elip)

  ## Ajuste dos valores de Theta calculado 0-180 para 0-360 ##
  Theta180Elip, Theta360Elip = [], []
  R180Elip, R360Elip = [], []
  Theta180Cont, Theta360Cont = [], []
  R180Cont, R360Cont = [], []

  for i in range(0,round(len(Theta_cont)/2)+2):
    Theta180Cont.append(Theta_cont[i])
    R180Cont.append(R_cont[i])

  for i in range(round(len(Theta_cont)/2)-1,len(Theta_cont)):
     Theta360Cont.append(Theta_cont[i])
     R360Cont.append(R_cont[i])
     
  Theta180Cont = np.asarray(Theta180Cont)
  Theta360Cont = np.asarray(Theta360Cont)
  R180Cont = np.asarray(R180Cont)
  R360Cont = np.asarray(R360Cont)

  Theta180_cont = -np.asarray(Theta180Cont)+360
  Theta360_cont = np.asarray(Theta360Cont)
  R180_cont = np.asarray(R180Cont)
  R360_cont =np.asarray(R360Cont)
  Theta_cont_final = np.concatenate([Theta180_cont,Theta360_cont])
  R_cont_final = np.concatenate([R180_cont,R360_cont])


  for i in range(0,round(len(Theta_elip)/2)):
    Theta180Elip.append(Theta_elip[i])
    R180Elip.append(R_elip[i])

  for i in range(round(len(Theta_elip)/2),len(Theta_elip)):
    Theta360Elip.append(Theta_elip[i])
    R360Elip.append(R_elip[i])

  Theta180_elip = -np.asarray(Theta180Elip)+360
  Theta360_elip = np.asarray(Theta360Elip)
  R180_elip = np.asarray(R180Elip)
  R360_elip =np.asarray(R360Elip)
  Theta_elip_final = np.concatenate([Theta180_elip,Theta360_elip])
  R_elip_final = np.concatenate([R180_elip, R360_elip])

  
  ## Ordenação dos valores de Theta ##
  Theta = pd.DataFrame(Theta_cont_final, columns = ['Theta'])
  R = pd.DataFrame(R_cont_final, columns = ['Raio'])
  Total = pd.concat([Theta, R], axis=1)
  Total = Total.fillna(method = 'backfill',axis = 1)
  Tec = Total['Theta'].apply(lambda x : get_int(x))
  Tec = pd.DataFrame(Tec, columns = ['Theta'])
  Total = pd.concat([Tec,R], axis=1)
  Total_cont = Total.sort_values(by=['Theta'])
  Total_cont['Theta'] = Total_cont['Theta']*10**-6
  
  Theta = pd.DataFrame(Theta_elip_final, columns = ['Theta'])
  R = pd.DataFrame(R_elip_final, columns = ['Raio'])
  Total = pd.concat([Theta, R], axis=1)
  Total = Total.fillna(method = 'backfill',axis = 1)
  Tec = Total['Theta'].apply(lambda x : get_int(x))
  Tec = pd.DataFrame(Tec, columns = ['Theta'])
  Total = pd.concat([Tec,R], axis=1)
  Total_elip = Total.sort_values(by=['Theta'])
  Total_elip['Theta'] = Total_elip['Theta']*10**-6

  ## Cálculo das diferença entre as áreas dos dois gráficos ##
  X = Total_elip['Theta']
  Y = Total_elip['Raio']
  Area_elip = auc(X,Y)
  X = Total_cont['Theta']
  Y = Total_cont['Raio']
  Area_cont = auc(X,Y)
  Dif_area = abs(Area_cont - Area_elip)

  AmpSup = abs(Total_elip['Raio'].max() - Total_cont['Raio'].max())
  AmpInf = abs(Total_elip['Raio'].min() - Total_cont['Raio'].min())


  ## Cálculo de amplitude absoluta ##
  Max_raio_cont = Total_cont['Raio'].max()
  Min_raio_cont = Total_cont['Raio'].min()
  Raio_cont_med = Total_cont['Raio'].mean()
  Raio_cont_mdn = Total_cont['Raio'].median()
  Raio_cont_Q25 = Total_cont['Raio'].quantile(q=0.25)
  Raio_cont_Q75 = Total_cont['Raio'].quantile(q=0.75)
  Raio_cont_mod = Total_cont['Raio'].mode()[0]
  Raio_cont_sta = Total_cont['Raio'].mad()
  Raio_cont_geo = stats2.gmean(Total_cont['Raio'])
  Raio_cont_har = stats2.hmean(Total_cont['Raio'])
  Amp_cont = Max_raio_cont - Min_raio_cont
  Amp_cont_abs = Amp_cont/Max_raio_cont

  print('---------------------------------------------------------------------------')
  final_time = time.time() - start_time
  print('Imagem: {} // Tempo de Código: {} min'.format(img,(final_time/60)))
  
  return Rmed, Rvar, Rstd, Gmed, Gvar, Gstd, Bmed, Bvar, Bstd, Graymed, Grayvar, Graystd, raio, area, perimetro, assymx, assymy, Dif_area, Amp_cont_abs, Rmax,Rmin, Rmdn, RQ25, RQ75, Rmod, Ramp, Rsta, Rgeo, Rhar, Gmax, Gmin, Gmdn, GQ25, GQ75, Gmod, Gamp, Gsta, Ggeo, Ghar, Bmax, Bmin, Bmdn, BQ25, BQ75, Bmod, Bamp, Bsta, Bgeo, Bhar, Graymax, Graymin, Graymdn, GrayQ25, GrayQ75, Graymod, Grayamp, Graysta, Graygeo, Grayhar, Amp_cont, Max_raio_cont, Min_raio_cont, Raio_cont_med, Raio_cont_mdn, Raio_cont_Q25, Raio_cont_Q75, Raio_cont_mod, Raio_cont_sta, Raio_cont_geo, Raio_cont_har, AmpSup, AmpInf, Rmode_len, Gmode_len, Bmode_len, Graymode_len, Rmed_adap, Rvar_adap, Rstd_adap, Rmax_adap, Rmin_adap, Rmdn_adap, RQ25_adap, RQ75_adap, Rmod_adap, Ramp_adap, Rsta_adap, Rgeo_adap, Rhar_adap, Rmode_adap_len, Gmed_adap, Gvar_adap, Gstd_adap, Gmax_adap, Gmin_adap, Gmdn_adap, GQ25_adap, GQ75_adap, Gmod_adap, Gamp_adap, Gsta_adap, Ggeo_adap, Ghar_adap, Gmode_adap_len, Bmed_adap, Bvar_adap, Bstd_adap, Bmax_adap, Bmin_adap, Bmdn_adap, BQ25_adap, BQ75_adap, Bmod_adap, Bamp_adap, Bsta_adap, Bgeo_adap, Bhar_adap,  Bmode_adap_len, Graymed_adap, Grayvar_adap, Graystd_adap, Graymax_adap, Graymin_adap, Graymdn_adap, GrayQ25_adap, GrayQ75_adap, Graymod_adap,  Grayamp_adap, Graysta_adap, Graygeo_adap, Grayhar_adap, Graymode_adap_len, HSL_Hmed, HSL_Hvar, HSL_Hstd, HSL_Hmax, HSL_Hmin, HSL_Hmdn, HSL_HQ25, HSL_HQ75, HSL_Hmod, HSL_Hamp, HSL_Hsta, HSL_Hgeo, HSL_Hhar, HSL_Hmode_len, HSL_Smed, HSL_Svar, HSL_Sstd, HSL_Smax, HSL_Smin, HSL_Smdn, HSL_SQ25, HSL_SQ75, HSL_Smod, HSL_Samp, HSL_Ssta, HSL_Sgeo, HSL_Shar, HSL_Smode_len, HSL_Lmed, HSL_Lvar, HSL_Lstd, HSL_Lmax, HSL_Lmin, HSL_Lmdn, HSL_LQ25, HSL_LQ75, HSL_Lmod, HSL_Lamp, HSL_Lsta, HSL_Lgeo, HSL_Lhar, HSL_Lmode_len, HSV_Hmed, HSV_Hvar, HSV_Hstd, HSV_Hmax, HSV_Hmin, HSV_Hmdn, HSV_HQ25, HSV_HQ75, HSV_Hmod, HSV_Hamp, HSV_Hsta, HSV_Hgeo, HSV_Hhar, HSV_Hmode_len, HSV_Smed, HSV_Svar, HSV_Sstd, HSV_Smax, HSV_Smin, HSV_Smdn, HSV_SQ25, HSV_SQ75, HSV_Smod, HSV_Samp, HSV_Ssta, HSV_Sgeo, HSV_Shar, HSV_Smode_len, HSV_Vmed, HSV_Vvar, HSV_Vstd, HSV_Vmax, HSV_Vmin, HSV_Vmdn, HSV_VQ25, HSV_VQ75, HSV_Vmod, HSV_Vamp, HSV_Vsta, HSV_Vgeo, HSV_Vhar, HSV_Vmode_len, LAB_Lmed, LAB_Lvar, LAB_Lstd, LAB_Lmax, LAB_Lmin, LAB_Lmdn, LAB_LQ25, LAB_LQ75, LAB_Lmod, LAB_Lamp, LAB_Lsta, LAB_Lgeo, LAB_Lhar, LAB_Lmode_len, LAB_Amed, LAB_Avar, LAB_Astd, LAB_Amax, LAB_Amin, LAB_Amdn, LAB_AQ25, LAB_AQ75, LAB_Amod, LAB_Aamp, LAB_Asta, LAB_Ageo, LAB_Ahar, LAB_Amode_len, LAB_Bmed, LAB_Bvar, LAB_Bstd, LAB_Bmax, LAB_Bmin, LAB_Bmdn, LAB_BQ25, LAB_BQ75, LAB_Bmod, LAB_Bamp, LAB_Bsta, LAB_Bgeo, LAB_Bhar, LAB_Bmode_len, LUV_Lmed, LUV_Lvar, LUV_Lstd, LUV_Lmax, LUV_Lmin, LUV_Lmdn, LUV_LQ25, LUV_LQ75, LUV_Lmod, LUV_Lamp, LUV_Lsta, LUV_Lgeo, LUV_Lhar, LUV_Lmode_len, LUV_Umed, LUV_Uvar, LUV_Ustd, LUV_Umax, LUV_Umin, LUV_Umdn, LUV_UQ25, LUV_UQ75, LUV_Umod, LUV_Uamp, LUV_Usta, LUV_Ugeo, LUV_Uhar, LUV_Umode_len, LUV_Vmed, LUV_Vvar, LUV_Vstd, LUV_Vmax, LUV_Vmin, LUV_Vmdn, LUV_VQ25, LUV_VQ75, LUV_Vmod, LUV_Vamp, LUV_Vsta, LUV_Vgeo, LUV_Vhar, LUV_Vmode_len

# Manipulação do Dataset e ajuste para retirada de características

In [4]:
doc_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/TCC/PH2/Dataset/PH2_Separado/PH2_dataset.xlsx',
                      skiprows = 12) # Pulou-se 12 linhas devido a forma como está disposta o arquivo Excel
doc_data.head()

,Image Name,Histological Diagnosis,Common Nevus,Atypical Nevus,Melanoma,Asymmetry\n(0/1/2),Pigment Network\n(AT/T),Dots/Globules\n(A/AT/T),Streaks\n(A/P),Regression Areas\n(A/P),Blue-Whitish Veil\n(A/P),White,Red,Light-Brown,Dark-Brown,Blue-Gray,Black
0,IMD003,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,NaN,X,NaN,NaN
1,IMD009,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,NaN,NaN,NaN
2,IMD016,NaN,X,NaN,NaN,0,T,T,A,A,A,NaN,NaN,X,X,NaN,NaN
3,IMD022,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,NaN,NaN,NaN
4,IMD024,NaN,X,NaN,NaN,0,T,A,A,A,A,NaN,NaN,X,X,NaN,NaN


In [5]:
doc_data.columns = ['img_id', 'histo_diagnosis', 'Common_Nevus',
                    'Atypical_Nevus', 'Melanoma', 'asymmetry',
                    'pigment_net', 'dots_glob', 'streaks', 'regressionA',
                    'blue_whit', 'Color_W', 'Color_R', 'Color_L_B', 'Color_D_B', 'Color_B_G', 'Color_B']

In [6]:
# A ausência de valor, ou seja, NaN indica que aquele local não foi marcado na planilha, então, vamos preencher com 0
doc_data = doc_data.fillna(0)

# Vamos mapear agora todos os espaços com 'X' e escrever a resposta binária 1
x_mapping = {"X": 1, 0: 0}
doc_data['Common_Nevus']   = doc_data['Common_Nevus'].map(x_mapping)
doc_data['Atypical_Nevus'] = doc_data['Atypical_Nevus'].map(x_mapping)
doc_data['Melanoma']       = doc_data['Melanoma'].map(x_mapping)
doc_data['Color_W']        = doc_data['Color_W'].map(x_mapping)
doc_data['Color_R']        = doc_data['Color_R'].map(x_mapping)
doc_data['Color_L_B']      = doc_data['Color_L_B'].map(x_mapping)
doc_data['Color_D_B']      = doc_data['Color_D_B'].map(x_mapping)
doc_data['Color_B_G']      = doc_data['Color_B_G'].map(x_mapping)
doc_data['Color_B']        = doc_data['Color_B'].map(x_mapping)
doc_data.head(5)

,img_id,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma,asymmetry,pigment_net,dots_glob,streaks,regressionA,blue_whit,Color_W,Color_R,Color_L_B,Color_D_B,Color_B_G,Color_B
0,IMD003,0,1,0,0,0,T,A,A,A,A,0,0,0,1,0,0
1,IMD009,0,1,0,0,0,T,A,A,A,A,0,0,1,0,0,0
2,IMD016,0,1,0,0,0,T,T,A,A,A,0,0,1,1,0,0
3,IMD022,0,1,0,0,0,T,A,A,A,A,0,0,1,0,0,0
4,IMD024,0,1,0,0,0,T,A,A,A,A,0,0,1,1,0,0


In [7]:
print('Valores na Coluna Pigment_net são: {}'.format(doc_data['pigment_net'].unique()))
print('Valores na Coluna Dots_Glob são: {}'.format(doc_data['dots_glob'].unique()))
print('Valores na Coluna Streaks são: {}'.format(doc_data['streaks'].unique()))
print('Valores na Coluna RegressionA são: {}'.format(doc_data['regressionA'].unique()))
print('Valores na Coluna Blue_Whit são: {}'.format(doc_data['blue_whit'].unique()))

Valores na Coluna Pigment_net são: ['T' 'AT']
Valores na Coluna Dots_Glob são: ['A' 'T' 'AT']
Valores na Coluna Streaks são: ['A' 'P']
Valores na Coluna RegressionA são: ['A' 'P']
Valores na Coluna Blue_Whit são: ['A' 'P']


In [8]:
AT_mapping = {'A': 'Ausente' , 'T': 'Típico' , 'AT': 'Atípico'}
P_mapping = {'A':'Ausente', 'P': 'Presente'}
doc_data['pigment_net'] = doc_data['pigment_net'].map(AT_mapping)
doc_data['dots_glob']   = doc_data['dots_glob'].map(AT_mapping)
doc_data['streaks']     = doc_data['streaks'].map(P_mapping)
doc_data['regressionA'] = doc_data['regressionA'].map(P_mapping)
doc_data['blue_whit']   = doc_data['blue_whit'].map(P_mapping)

In [9]:
doc_data.head()

,img_id,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma,asymmetry,pigment_net,dots_glob,streaks,regressionA,blue_whit,Color_W,Color_R,Color_L_B,Color_D_B,Color_B_G,Color_B
0,IMD003,0,1,0,0,0,Típico,Ausente,Ausente,Ausente,Ausente,0,0,0,1,0,0
1,IMD009,0,1,0,0,0,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0
2,IMD016,0,1,0,0,0,Típico,Típico,Ausente,Ausente,Ausente,0,0,1,1,0,0
3,IMD022,0,1,0,0,0,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0
4,IMD024,0,1,0,0,0,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,1,0,0


In [10]:
CN = doc_data['Common_Nevus']*1
AN = doc_data['Atypical_Nevus']*2
MN = doc_data['Melanoma']*3
dic_class = {1: 'Common Nevus', 2: 'Atypical Nevus', 3:'Melanoma'}
big_class = (CN+AN+MN).map(dic_class)
doc_data['Class'] = big_class

In [11]:
asy_dic = {0: 'Simétrico' , 1: 'Parcialmente A.', 2: 'Totalmente A.'}
doc_data['asymmetry'] = doc_data['asymmetry'].map(asy_dic)

In [12]:
doc_data.head()

,img_id,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma,asymmetry,pigment_net,dots_glob,streaks,regressionA,blue_whit,Color_W,Color_R,Color_L_B,Color_D_B,Color_B_G,Color_B,Class
0,IMD003,0,1,0,0,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,0,1,0,0,Common Nevus
1,IMD009,0,1,0,0,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0,Common Nevus
2,IMD016,0,1,0,0,Simétrico,Típico,Típico,Ausente,Ausente,Ausente,0,0,1,1,0,0,Common Nevus
3,IMD022,0,1,0,0,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0,Common Nevus
4,IMD024,0,1,0,0,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,1,0,0,Common Nevus


In [13]:
# Agora que a planilha foi padronizada e configurada em valores binários, vamos dividir em características de teste e resultado
result = doc_data[['Class', 'histo_diagnosis', 'Common_Nevus','Atypical_Nevus', 'Melanoma']]
doc_data = doc_data.drop(['Class','histo_diagnosis','Common_Nevus','Atypical_Nevus','Melanoma'],axis=1)

In [14]:
result.head(3)

,Class,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma
0,Common Nevus,0,1,0,0
1,Common Nevus,0,1,0,0
2,Common Nevus,0,1,0,0


In [15]:
doc_data.head(3)

,img_id,asymmetry,pigment_net,dots_glob,streaks,regressionA,blue_whit,Color_W,Color_R,Color_L_B,Color_D_B,Color_B_G,Color_B
0,IMD003,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,0,1,0,0
1,IMD009,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0
2,IMD016,Simétrico,Típico,Típico,Ausente,Ausente,Ausente,0,0,1,1,0,0


In [16]:
full_doc_data = pd.concat([doc_data, result], axis=1, join='inner')
full_doc_data

,img_id,asymmetry,pigment_net,dots_glob,streaks,regressionA,blue_whit,Color_W,Color_R,Color_L_B,Color_D_B,Color_B_G,Color_B,Class,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma
0,IMD003,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,0,1,0,0,Common Nevus,0,1,0,0
1,IMD009,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0,Common Nevus,0,1,0,0
2,IMD016,Simétrico,Típico,Típico,Ausente,Ausente,Ausente,0,0,1,1,0,0,Common Nevus,0,1,0,0
3,IMD022,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0,Common Nevus,0,1,0,0
4,IMD024,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,1,0,0,Common Nevus,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IMD424,Totalmente A.,Atípico,Atípico,Presente,Ausente,Presente,0,0,1,1,1,1,Melanoma,Melanoma,0,0,1
196,IMD425,Totalmente A.,Atípico,Atípico,Ausente,Presente,Presente,0,0,1,1,1,1,Melanoma,Melanoma,0,0,1
197,IMD426,Totalmente A.,Atípico,Ausente,Ausente,Presente,Ausente,0,0,1,1,0,0,Melanoma,Melanoma,0,0,1
198,IMD429,Simétrico,Atípico,Ausente,Presente,Ausente,Presente,0,0,1,1,1,1,Melanoma,0,0,0,1


# Extração das características da imagem

In [17]:
# Agora, vamos criar um DataFrame, utilizando somente as imagens e dados coletados dela
img_data = pd.DataFrame()
img_data['img_id'] = doc_data['img_id']
img_data.head()

,img_id
0,IMD003
1,IMD009
2,IMD016
3,IMD022
4,IMD024


In [18]:
#img_data = img_data.sample(5)
img_data

,img_id
0,IMD003
1,IMD009
2,IMD016
3,IMD022
4,IMD024
...,...
195,IMD424
196,IMD425
197,IMD426
198,IMD429


In [19]:
all_features = img_data['img_id'].apply(lambda x : get_features(x))

---------------------------------------------------------------------------
Imagem: IMD003 // Tempo de Código: 0.39485006729761757 min
---------------------------------------------------------------------------
Imagem: IMD009 // Tempo de Código: 0.5308547973632812 min
---------------------------------------------------------------------------
Imagem: IMD016 // Tempo de Código: 0.6940737922986349 min
---------------------------------------------------------------------------
Imagem: IMD022 // Tempo de Código: 1.001417609055837 min
---------------------------------------------------------------------------
Imagem: IMD024 // Tempo de Código: 1.1234231074651082 min
---------------------------------------------------------------------------
Imagem: IMD025 // Tempo de Código: 1.310234272480011 min
---------------------------------------------------------------------------
Imagem: IMD035 // Tempo de Código: 1.4321884711583455 min
---------------------------------------------------------------

In [20]:
all_features = np.asarray(all_features)

In [21]:
len(all_features[0])

300

In [22]:
Rmed, Rvar, Rstd = [],[],[]
Gmed, Gvar, Gstd = [],[],[]
Bmed, Bvar, Bstd = [],[],[]
Graymed, Grayvar, Graystd = [],[],[]
raio, area, perimetro = [],[],[]
Rmax, Rmin, Rmdn, RQ25, RQ75, Rmod, Ramp, Rsta, Rgeo, Rhar = [],[],[],[],[],[],[],[],[],[]
Gmax, Gmin, Gmdn, GQ25, GQ75, Gmod, Gamp, Gsta, Ggeo, Ghar = [],[],[],[],[],[],[],[],[],[]
Bmax, Bmin, Bmdn, BQ25, BQ75, Bmod, Bamp, Bsta, Bgeo, Bhar = [],[],[],[],[],[],[],[],[],[]
Graymax, Graymin, Graymdn, GrayQ25, GrayQ75, Graymod, Grayamp, Graysta, Graygeo, Grayhar = [],[],[],[],[],[],[],[],[],[]
Raioamp, Raiomax, Raiomin, Raiomed, Raiomdn, RaioQ25, RaioQ75, Raiomod, Raiosta, Raiogeo, Raiohar = [],[],[],[],[],[],[],[],[],[],[]
assymx, assymy, Dif_area, Amp_cont_abs = [],[],[],[]
AmpSup, AmpInf =[],[]
Rmode_len, Gmode_len, Bmode_len, Graymode_len = [], [], [], []
Rmed_adap, Rvar_adap, Rstd_adap, Rmax_adap, Rmin_adap, Rmdn_adap, RQ25_adap = [],[],[],[],[],[],[]
RQ75_adap, Rmod_adap, Ramp_adap, Rsta_adap, Rgeo_adap, Rhar_adap, Rmode_adap_len  = [],[],[],[],[],[],[]
Gmed_adap, Gvar_adap, Gstd_adap, Gmax_adap, Gmin_adap, Gmdn_adap, GQ25_adap  = [],[],[],[],[],[],[]
GQ75_adap, Gmod_adap, Gamp_adap, Gsta_adap, Ggeo_adap, Ghar_adap, Gmode_adap_len  = [],[],[],[],[],[],[]
Bmed_adap, Bvar_adap, Bstd_adap, Bmax_adap, Bmin_adap, Bmdn_adap, BQ25_adap  = [],[],[],[],[],[],[]
BQ75_adap, Bmod_adap, Bamp_adap, Bsta_adap, Bgeo_adap, Bhar_adap,  Bmode_adap_len  = [],[],[],[],[],[],[]
Graymed_adap, Grayvar_adap, Graystd_adap, Graymax_adap, Graymin_adap, Graymdn_adap, GrayQ25_adap  = [],[],[],[],[],[],[]
GrayQ75_adap, Graymod_adap,  Grayamp_adap, Graysta_adap, Graygeo_adap, Grayhar_adap, Graymode_adap_len  = [],[],[],[],[],[],[]
HSL_Hmed, HSL_Hvar, HSL_Hstd, HSL_Hmax, HSL_Hmin, HSL_Hmdn, HSL_HQ25, HSL_HQ75, HSL_Hmod, HSL_Hamp, HSL_Hsta, HSL_Hgeo, HSL_Hhar, HSL_Hmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
HSL_Smed, HSL_Svar, HSL_Sstd, HSL_Smax, HSL_Smin, HSL_Smdn, HSL_SQ25, HSL_SQ75, HSL_Smod, HSL_Samp, HSL_Ssta, HSL_Sgeo, HSL_Shar, HSL_Smode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
HSL_Lmed, HSL_Lvar, HSL_Lstd, HSL_Lmax, HSL_Lmin, HSL_Lmdn, HSL_LQ25, HSL_LQ75, HSL_Lmod, HSL_Lamp, HSL_Lsta, HSL_Lgeo, HSL_Lhar, HSL_Lmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
HSV_Hmed, HSV_Hvar, HSV_Hstd, HSV_Hmax, HSV_Hmin, HSV_Hmdn, HSV_HQ25, HSV_HQ75, HSV_Hmod, HSV_Hamp, HSV_Hsta, HSV_Hgeo, HSV_Hhar, HSV_Hmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
HSV_Smed, HSV_Svar, HSV_Sstd, HSV_Smax, HSV_Smin, HSV_Smdn, HSV_SQ25, HSV_SQ75, HSV_Smod, HSV_Samp, HSV_Ssta, HSV_Sgeo, HSV_Shar, HSV_Smode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
HSV_Vmed, HSV_Vvar, HSV_Vstd, HSV_Vmax, HSV_Vmin, HSV_Vmdn, HSV_VQ25, HSV_VQ75, HSV_Vmod, HSV_Vamp, HSV_Vsta, HSV_Vgeo, HSV_Vhar, HSV_Vmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
LAB_Lmed, LAB_Lvar, LAB_Lstd, LAB_Lmax, LAB_Lmin, LAB_Lmdn, LAB_LQ25, LAB_LQ75, LAB_Lmod, LAB_Lamp, LAB_Lsta, LAB_Lgeo, LAB_Lhar, LAB_Lmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
LAB_Amed, LAB_Avar, LAB_Astd, LAB_Amax, LAB_Amin, LAB_Amdn, LAB_AQ25, LAB_AQ75, LAB_Amod, LAB_Aamp, LAB_Asta, LAB_Ageo, LAB_Ahar, LAB_Amode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
LAB_Bmed, LAB_Bvar, LAB_Bstd, LAB_Bmax, LAB_Bmin, LAB_Bmdn, LAB_BQ25, LAB_BQ75, LAB_Bmod, LAB_Bamp, LAB_Bsta, LAB_Bgeo, LAB_Bhar, LAB_Bmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
LUV_Lmed, LUV_Lvar, LUV_Lstd, LUV_Lmax, LUV_Lmin, LUV_Lmdn, LUV_LQ25, LUV_LQ75, LUV_Lmod, LUV_Lamp, LUV_Lsta, LUV_Lgeo, LUV_Lhar, LUV_Lmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
LUV_Umed, LUV_Uvar, LUV_Ustd, LUV_Umax, LUV_Umin, LUV_Umdn, LUV_UQ25, LUV_UQ75, LUV_Umod, LUV_Uamp, LUV_Usta, LUV_Ugeo, LUV_Uhar, LUV_Umode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
LUV_Vmed, LUV_Vvar, LUV_Vstd, LUV_Vmax, LUV_Vmin, LUV_Vmdn, LUV_VQ25, LUV_VQ75, LUV_Vmod, LUV_Vamp, LUV_Vsta, LUV_Vgeo, LUV_Vhar, LUV_Vmode_len  = [],[],[],[],[],[],[],[],[],[],[],[],[],[]


for i,col in enumerate(all_features):
  Rmed.append(all_features[i][0])
  Rvar.append(all_features[i][1])
  Rstd.append(all_features[i][2])
  Gmed.append(all_features[i][3])
  Gvar.append(all_features[i][4])
  Gstd.append(all_features[i][5])
  Bmed.append(all_features[i][6])
  Bvar.append(all_features[i][7])
  Bstd.append(all_features[i][8])
  Graymed.append(all_features[i][9])
  Grayvar.append(all_features[i][10])
  Graystd.append(all_features[i][11])
  raio.append(all_features[i][12])
  area.append(all_features[i][13])
  perimetro.append(all_features[i][14])
  assymx.append(all_features[i][15])
  assymy.append(all_features[i][16])
  Dif_area.append(all_features[i][17])
  Amp_cont_abs.append(all_features[i][18])
  Rmax.append(all_features[i][19])
  Rmin.append(all_features[i][20])
  Rmdn.append(all_features[i][21])
  RQ25.append(all_features[i][22])
  RQ75.append(all_features[i][23])
  Rmod.append(all_features[i][24])
  Ramp.append(all_features[i][25])
  Rsta.append(all_features[i][26])
  Rgeo.append(all_features[i][27])
  Rhar.append(all_features[i][28])
  Gmax.append(all_features[i][29])
  Gmin.append(all_features[i][30])
  Gmdn.append(all_features[i][31])
  GQ25.append(all_features[i][32])
  GQ75.append(all_features[i][33])
  Gmod.append(all_features[i][34])
  Gamp.append(all_features[i][35])
  Gsta.append(all_features[i][36])
  Ggeo.append(all_features[i][37])
  Ghar.append(all_features[i][38])
  Bmax.append(all_features[i][39])
  Bmin.append(all_features[i][40])
  Bmdn.append(all_features[i][41])
  BQ25.append(all_features[i][42])
  BQ75.append(all_features[i][43])
  Bmod.append(all_features[i][44])
  Bamp.append(all_features[i][45])
  Bsta.append(all_features[i][46])
  Bgeo.append(all_features[i][47])
  Bhar.append(all_features[i][48])
  Graymax.append(all_features[i][49])
  Graymin.append(all_features[i][50])
  Graymdn.append(all_features[i][51])
  GrayQ25.append(all_features[i][52])
  GrayQ75.append(all_features[i][53])
  Graymod.append(all_features[i][54])
  Grayamp.append(all_features[i][55])
  Graysta.append(all_features[i][56])
  Graygeo.append(all_features[i][57])
  Grayhar.append(all_features[i][58])
  Raioamp.append(all_features[i][59])
  Raiomax.append(all_features[i][60])
  Raiomin.append(all_features[i][61])
  Raiomed.append(all_features[i][62])
  Raiomdn.append(all_features[i][63])
  RaioQ25.append(all_features[i][64])
  RaioQ75.append(all_features[i][65])
  Raiomod.append(all_features[i][66])
  Raiosta.append(all_features[i][67])
  Raiogeo.append(all_features[i][68])
  Raiohar.append(all_features[i][69])
  AmpSup.append(all_features[i][70])
  AmpInf.append(all_features[i][71])
  Rmode_len.append(all_features[i][72])
  Gmode_len.append(all_features[i][73])
  Bmode_len.append(all_features[i][74])
  Graymode_len.append(all_features[i][75])
  Rmed_adap.append(all_features[i][76])
  Rvar_adap.append(all_features[i][77])
  Rstd_adap.append(all_features[i][78])
  Rmax_adap.append(all_features[i][79])
  Rmin_adap.append(all_features[i][80])
  Rmdn_adap.append(all_features[i][81])
  RQ25_adap.append(all_features[i][82])
  RQ75_adap.append(all_features[i][83])
  Rmod_adap.append(all_features[i][84])
  Ramp_adap.append(all_features[i][85])
  Rsta_adap.append(all_features[i][86])
  Rgeo_adap.append(all_features[i][87])
  Rhar_adap.append(all_features[i][88])
  Rmode_adap_len.append(all_features[i][89])
  Gmed_adap.append(all_features[i][90])
  Gvar_adap.append(all_features[i][91])
  Gstd_adap.append(all_features[i][92])
  Gmax_adap.append(all_features[i][93])
  Gmin_adap.append(all_features[i][94])
  Gmdn_adap.append(all_features[i][95])
  GQ25_adap.append(all_features[i][96])
  GQ75_adap.append(all_features[i][97])
  Gmod_adap.append(all_features[i][98])
  Gamp_adap.append(all_features[i][99])
  Gsta_adap.append(all_features[i][100])
  Ggeo_adap.append(all_features[i][101])
  Ghar_adap.append(all_features[i][102])
  Gmode_adap_len.append(all_features[i][103])
  Bmed_adap.append(all_features[i][104])
  Bvar_adap.append(all_features[i][105])
  Bstd_adap.append(all_features[i][106])
  Bmax_adap.append(all_features[i][107])
  Bmin_adap.append(all_features[i][108])
  Bmdn_adap.append(all_features[i][109])
  BQ25_adap.append(all_features[i][110])
  BQ75_adap.append(all_features[i][111])
  Bmod_adap.append(all_features[i][112])
  Bamp_adap.append(all_features[i][113])
  Bsta_adap.append(all_features[i][114])
  Bgeo_adap.append(all_features[i][115])
  Bhar_adap.append(all_features[i][116])
  Bmode_adap_len.append(all_features[i][117])
  Graymed_adap.append(all_features[i][118])
  Grayvar_adap.append(all_features[i][119])
  Graystd_adap.append(all_features[i][120])
  Graymax_adap.append(all_features[i][121])
  Graymin_adap.append(all_features[i][122])
  Graymdn_adap.append(all_features[i][123])
  GrayQ25_adap.append(all_features[i][124])
  GrayQ75_adap.append(all_features[i][125])
  Graymod_adap.append(all_features[i][126])
  Grayamp_adap.append(all_features[i][127])
  Graysta_adap.append(all_features[i][128])
  Graygeo_adap.append(all_features[i][129])
  Grayhar_adap.append(all_features[i][130])
  Graymode_adap_len.append(all_features[i][131])
  HSL_Hmed.append(all_features[i][132])
  HSL_Hvar.append(all_features[i][133])
  HSL_Hstd.append(all_features[i][134])
  HSL_Hmax.append(all_features[i][135])
  HSL_Hmin.append(all_features[i][136])
  HSL_Hmdn.append(all_features[i][137])
  HSL_HQ25.append(all_features[i][138])
  HSL_HQ75.append(all_features[i][139])
  HSL_Hmod.append(all_features[i][140])
  HSL_Hamp.append(all_features[i][141])
  HSL_Hsta.append(all_features[i][142])
  HSL_Hgeo.append(all_features[i][143])
  HSL_Hhar.append(all_features[i][144])
  HSL_Hmode_len.append(all_features[i][145])
  HSL_Smed.append(all_features[i][146])
  HSL_Svar.append(all_features[i][147])
  HSL_Sstd.append(all_features[i][148])
  HSL_Smax.append(all_features[i][149])
  HSL_Smin.append(all_features[i][150])
  HSL_Smdn.append(all_features[i][151])
  HSL_SQ25.append(all_features[i][152])
  HSL_SQ75.append(all_features[i][153])
  HSL_Smod.append(all_features[i][154])
  HSL_Samp.append(all_features[i][155])
  HSL_Ssta.append(all_features[i][156])
  HSL_Sgeo.append(all_features[i][157])
  HSL_Shar.append(all_features[i][158])
  HSL_Smode_len.append(all_features[i][159])
  HSL_Lmed.append(all_features[i][160])
  HSL_Lvar.append(all_features[i][161])
  HSL_Lstd.append(all_features[i][162])
  HSL_Lmax.append(all_features[i][163])
  HSL_Lmin.append(all_features[i][164])
  HSL_Lmdn.append(all_features[i][165])
  HSL_LQ25.append(all_features[i][166])
  HSL_LQ75.append(all_features[i][167])
  HSL_Lmod.append(all_features[i][168])
  HSL_Lamp.append(all_features[i][169])
  HSL_Lsta.append(all_features[i][170])
  HSL_Lgeo.append(all_features[i][171])
  HSL_Lhar.append(all_features[i][172])
  HSL_Lmode_len.append(all_features[i][173])
  HSV_Hmed.append(all_features[i][174])
  HSV_Hvar.append(all_features[i][175])
  HSV_Hstd.append(all_features[i][176])
  HSV_Hmax.append(all_features[i][177])
  HSV_Hmin.append(all_features[i][178])
  HSV_Hmdn.append(all_features[i][179])
  HSV_HQ25.append(all_features[i][180])
  HSV_HQ75.append(all_features[i][181])
  HSV_Hmod.append(all_features[i][182])
  HSV_Hamp.append(all_features[i][183])
  HSV_Hsta.append(all_features[i][184])
  HSV_Hgeo.append(all_features[i][185])
  HSV_Hhar.append(all_features[i][186])
  HSV_Hmode_len.append(all_features[i][187])
  HSV_Smed.append(all_features[i][188])
  HSV_Svar.append(all_features[i][189])
  HSV_Sstd.append(all_features[i][190])
  HSV_Smax.append(all_features[i][191])
  HSV_Smin.append(all_features[i][192])
  HSV_Smdn.append(all_features[i][193])
  HSV_SQ25.append(all_features[i][194])
  HSV_SQ75.append(all_features[i][195])
  HSV_Smod.append(all_features[i][196])
  HSV_Samp.append(all_features[i][197])
  HSV_Ssta.append(all_features[i][198])
  HSV_Sgeo.append(all_features[i][199])
  HSV_Shar.append(all_features[i][200])
  HSV_Smode_len.append(all_features[i][201])
  HSV_Vmed.append(all_features[i][202])
  HSV_Vvar.append(all_features[i][203])
  HSV_Vstd.append(all_features[i][204])
  HSV_Vmax.append(all_features[i][205])
  HSV_Vmin.append(all_features[i][206])
  HSV_Vmdn.append(all_features[i][207])
  HSV_VQ25.append(all_features[i][208])
  HSV_VQ75.append(all_features[i][209])
  HSV_Vmod.append(all_features[i][210])
  HSV_Vamp.append(all_features[i][211])
  HSV_Vsta.append(all_features[i][212])
  HSV_Vgeo.append(all_features[i][213])
  HSV_Vhar.append(all_features[i][214])
  HSV_Vmode_len.append(all_features[i][215])
  LAB_Lmed.append(all_features[i][216])
  LAB_Lvar.append(all_features[i][217])
  LAB_Lstd.append(all_features[i][218])
  LAB_Lmax.append(all_features[i][219])
  LAB_Lmin.append(all_features[i][220])
  LAB_Lmdn.append(all_features[i][221])
  LAB_LQ25.append(all_features[i][222])
  LAB_LQ75.append(all_features[i][223])
  LAB_Lmod.append(all_features[i][224])
  LAB_Lamp.append(all_features[i][225])
  LAB_Lsta.append(all_features[i][226])
  LAB_Lgeo.append(all_features[i][227])
  LAB_Lhar.append(all_features[i][228])
  LAB_Lmode_len.append(all_features[i][229])
  LAB_Amed.append(all_features[i][230])
  LAB_Avar.append(all_features[i][231])
  LAB_Astd.append(all_features[i][232])
  LAB_Amax.append(all_features[i][233])
  LAB_Amin.append(all_features[i][234])
  LAB_Amdn.append(all_features[i][235])
  LAB_AQ25.append(all_features[i][236])
  LAB_AQ75.append(all_features[i][237])
  LAB_Amod.append(all_features[i][238])
  LAB_Aamp.append(all_features[i][239])
  LAB_Asta.append(all_features[i][240])
  LAB_Ageo.append(all_features[i][241])
  LAB_Ahar.append(all_features[i][242])
  LAB_Amode_len.append(all_features[i][243])
  LAB_Bmed.append(all_features[i][244])
  LAB_Bvar.append(all_features[i][245])
  LAB_Bstd.append(all_features[i][246])
  LAB_Bmax.append(all_features[i][247])
  LAB_Bmin.append(all_features[i][248])
  LAB_Bmdn.append(all_features[i][249])
  LAB_BQ25.append(all_features[i][250])
  LAB_BQ75.append(all_features[i][251])
  LAB_Bmod.append(all_features[i][252])
  LAB_Bamp.append(all_features[i][253])
  LAB_Bsta.append(all_features[i][254])
  LAB_Bgeo.append(all_features[i][255])
  LAB_Bhar.append(all_features[i][256])
  LAB_Bmode_len.append(all_features[i][257])
  LUV_Lmed.append(all_features[i][258])
  LUV_Lvar.append(all_features[i][259])
  LUV_Lstd.append(all_features[i][260])
  LUV_Lmax.append(all_features[i][261])
  LUV_Lmin.append(all_features[i][262])
  LUV_Lmdn.append(all_features[i][263])
  LUV_LQ25.append(all_features[i][264])
  LUV_LQ75.append(all_features[i][265])
  LUV_Lmod.append(all_features[i][266])
  LUV_Lamp.append(all_features[i][267])
  LUV_Lsta.append(all_features[i][268])
  LUV_Lgeo.append(all_features[i][269])
  LUV_Lhar.append(all_features[i][270])
  LUV_Lmode_len.append(all_features[i][271])
  LUV_Umed.append(all_features[i][272])
  LUV_Uvar.append(all_features[i][273])
  LUV_Ustd.append(all_features[i][274])
  LUV_Umax.append(all_features[i][275])
  LUV_Umin.append(all_features[i][276])
  LUV_Umdn.append(all_features[i][277])
  LUV_UQ25.append(all_features[i][278])
  LUV_UQ75.append(all_features[i][279])
  LUV_Umod.append(all_features[i][280])
  LUV_Uamp.append(all_features[i][281])
  LUV_Usta.append(all_features[i][282])
  LUV_Ugeo.append(all_features[i][283])
  LUV_Uhar.append(all_features[i][284])
  LUV_Umode_len.append(all_features[i][285])
  LUV_Vmed.append(all_features[i][286])
  LUV_Vvar.append(all_features[i][287])
  LUV_Vstd.append(all_features[i][288])
  LUV_Vmax.append(all_features[i][289])
  LUV_Vmin.append(all_features[i][290])
  LUV_Vmdn.append(all_features[i][291])
  LUV_VQ25.append(all_features[i][292])
  LUV_VQ75.append(all_features[i][293])
  LUV_Vmod.append(all_features[i][294])
  LUV_Vamp.append(all_features[i][295])
  LUV_Vsta.append(all_features[i][296])
  LUV_Vgeo.append(all_features[i][297])
  LUV_Vhar.append(all_features[i][298])
  LUV_Vmode_len.append(all_features[i][299])

Rmed = np.asarray(Rmed)
Rvar = np.asarray(Rvar)
Rstd = np.asarray(Rstd)
Gmed = np.asarray(Gmed)
Gvar = np.asarray(Gvar)
Gstd = np.asarray(Gstd)
Bmed = np.asarray(Bmed)
Bvar = np.asarray(Bvar)
Bstd = np.asarray(Bstd)
Graymed = np.asarray(Graymed)
Grayvar = np.asarray(Grayvar)
Graystd = np.asarray(Graystd)
raio = np.asarray(raio)
area = np.asarray(area)
perimetro = np.asarray(perimetro)
assymx = np.asarray(assymx)
assymy = np.asarray(assymy)
Dif_area = np.asarray(Dif_area)
Amp_cont_abs = np.asarray(Amp_cont_abs)
Rmax = np.asarray(Rmax)
Rmin = np.asarray(Rmin)
Rmdn = np.asarray(Rmdn)
RQ25 = np.asarray(RQ25)
RQ75 = np.asarray(RQ75)
Rmod = np.asarray(Rmod)
Ramp = np.asarray(Ramp)
Rsta = np.asarray(Rsta)
Rgeo = np.asarray(Rgeo)
Rhar = np.asarray(Rhar)
Gmax = np.asarray(Gmax)
Gmin = np.asarray(Gmin)
Gmdn = np.asarray(Gmdn)
GQ25 = np.asarray(GQ25)
GQ75 = np.asarray(GQ75)
Gmod = np.asarray(Gmod)
Gamp = np.asarray(Gamp)
Gsta = np.asarray(Gsta)
Ggeo = np.asarray(Ggeo)
Ghar = np.asarray(Ghar)
Bmax = np.asarray(Bmax)
Bmin = np.asarray(Bmin)
Bmdn = np.asarray(Bmdn)
BQ25 = np.asarray(BQ25)
BQ75 = np.asarray(BQ75)
Bmod = np.asarray(Bmod)
Bamp = np.asarray(Bamp)
Bsta = np.asarray(Bsta)
Bgeo = np.asarray(Bgeo)
Bhar = np.asarray(Bhar)
Graymax = np.asarray(Graymax)
Graymin = np.asarray(Graymin)
Graymdn = np.asarray(Graymdn)
GrayQ25 = np.asarray(GrayQ25)
GrayQ75 = np.asarray(GrayQ75)
Graymod = np.asarray(Graymod)
Grayamp = np.asarray(Grayamp)
Graysta = np.asarray(Graysta)
Graygeo = np.asarray(Graygeo)
Grayhar = np.asarray(Grayhar)
Graymax = np.asarray(Graymax)
Graymin = np.asarray(Graymin)
Graymdn = np.asarray(Graymdn)
GrayQ25 = np.asarray(GrayQ25)
GrayQ75 = np.asarray(GrayQ75)
Graymod = np.asarray(Graymod)
Grayamp = np.asarray(Grayamp)
Graysta = np.asarray(Graysta)
Graygeo = np.asarray(Graygeo)
Grayhar = np.asarray(Grayhar)
Raioamp = np.asarray(Raioamp)
Raiomax = np.asarray(Raiomax)
Raiomin = np.asarray(Raiomin)
Raiomed = np.asarray(Raiomed)
Raiomdn = np.asarray(Raiomdn)
RaioQ25 = np.asarray(RaioQ25)
RaioQ75 = np.asarray(RaioQ75)
Raiomod = np.asarray(Raiomod)
Raiosta = np.asarray(Raiosta)
Raiogeo = np.asarray(Raiogeo)
Raiohar = np.asarray(Raiohar)
AmpSup  = np.asarray(AmpSup)
AmpInf  = np.asarray(AmpInf)
Rmode_len  = np.asarray(Rmode_len)
Gmode_len  = np.asarray(Gmode_len)
Bmode_len  = np.asarray(Bmode_len)
Graymode_len  = np.asarray(Graymode_len)
Rmed_adap  = np.asarray(Rmed_adap)
Rvar_adap  = np.asarray(Rvar_adap)
Rstd_adap  = np.asarray(Rstd_adap)
Rmax_adap  = np.asarray(Rmax_adap)
Rmin_adap  = np.asarray(Rmin_adap)
Rmdn_adap  = np.asarray(Rmdn_adap)
RQ25_adap  = np.asarray(RQ25_adap)
RQ75_adap  = np.asarray(RQ75_adap)
Rmod_adap  = np.asarray(Rmod_adap)
Ramp_adap  = np.asarray(Ramp_adap)
Rsta_adap  = np.asarray(Rsta_adap) 
Rgeo_adap  = np.asarray(Rgeo_adap)
Rhar_adap  = np.asarray(Rhar_adap)
Rmode_adap_len  = np.asarray(Rmode_adap_len)
Gmed_adap  = np.asarray(Gmed_adap)
Gvar_adap  = np.asarray(Gvar_adap) 
Gstd_adap  = np.asarray(Gstd_adap)
Gmax_adap  = np.asarray(Gmax_adap)
Gmin_adap  = np.asarray(Gmin_adap)
Gmdn_adap  = np.asarray(Gmdn_adap)
GQ25_adap  = np.asarray(GQ25_adap)
GQ75_adap  = np.asarray(GQ75_adap)
Gmod_adap  = np.asarray(Gmod_adap)
Gamp_adap  = np.asarray(Gamp_adap)
Gsta_adap  = np.asarray(Gsta_adap)
Ggeo_adap  = np.asarray(Ggeo_adap)
Ghar_adap  = np.asarray(Ghar_adap)
Gmode_adap_len  = np.asarray(Gmode_adap_len)
Bmed_adap  = np.asarray(Bmed_adap)
Bvar_adap  = np.asarray(Bvar_adap)
Bstd_adap  = np.asarray(Bstd_adap) 
Bmax_adap  = np.asarray(Bmax_adap) 
Bmin_adap  = np.asarray(Bmin_adap) 
Bmdn_adap  = np.asarray(Bmdn_adap) 
BQ25_adap  = np.asarray(BQ25_adap) 
BQ75_adap  = np.asarray(BQ75_adap)
Bmod_adap  = np.asarray(Bmod_adap)
Bamp_adap  = np.asarray(Bamp_adap)
Bsta_adap  = np.asarray(Bsta_adap)
Bgeo_adap  = np.asarray(Bgeo_adap)
Bhar_adap  = np.asarray(Bhar_adap)
Bmode_adap_len  = np.asarray(Bmode_adap_len)
Graymed_adap  = np.asarray(Graymed_adap)
Grayvar_adap  = np.asarray(Grayvar_adap) 
Graystd_adap  = np.asarray(Graystd_adap) 
Graymax_adap  = np.asarray(Graymax_adap)
Graymin_adap  = np.asarray(Graymin_adap)
Graymdn_adap  = np.asarray(Graymdn_adap )
GrayQ25_adap  = np.asarray(GrayQ25_adap)
GrayQ75_adap  = np.asarray(GrayQ75_adap)
Graymod_adap  = np.asarray(Graymod_adap)
Grayamp_adap  = np.asarray(Grayamp_adap)
Graysta_adap  = np.asarray(Graysta_adap)
Graygeo_adap  = np.asarray(Graygeo_adap)
Grayhar_adap  = np.asarray(Grayhar_adap)
Graymode_adap_len   = np.asarray(Graymode_adap_len)
HSL_Hmed = np.asarray(HSL_Hmed)
HSL_Hvar = np.asarray(HSL_Hvar)
HSL_Hstd = np.asarray(HSL_Hstd)
HSL_Hmax = np.asarray(HSL_Hmax)
HSL_Hmin = np.asarray(HSL_Hmin)
HSL_Hmdn = np.asarray(HSL_Hmdn)
HSL_HQ25 = np.asarray(HSL_HQ25)
HSL_HQ75 = np.asarray(HSL_HQ75)
HSL_Hmod = np.asarray(HSL_Hmod)
HSL_Hamp = np.asarray(HSL_Hamp)
HSL_Hsta = np.asarray(HSL_Hsta)
HSL_Hgeo = np.asarray(HSL_Hgeo)
HSL_Hhar = np.asarray(HSL_Hhar)
HSL_Hmode_len = np.asarray(HSL_Hmode_len)
HSL_Smed = np.asarray(HSL_Smed)
HSL_Svar = np.asarray(HSL_Svar)
HSL_Sstd = np.asarray(HSL_Sstd)
HSL_Smax = np.asarray(HSL_Smax)
HSL_Smin = np.asarray(HSL_Smin)
HSL_Smdn = np.asarray(HSL_Smdn)
HSL_SQ25 = np.asarray(HSL_SQ25)
HSL_SQ75 = np.asarray(HSL_SQ75)
HSL_Smod = np.asarray(HSL_Smod)
HSL_Samp = np.asarray(HSL_Samp)
HSL_Ssta = np.asarray(HSL_Ssta)
HSL_Sgeo = np.asarray(HSL_Sgeo)
HSL_Shar = np.asarray(HSL_Shar)
HSL_Smode_len = np.asarray(HSL_Smode_len)
HSL_Lmed = np.asarray(HSL_Lmed)
HSL_Lvar = np.asarray(HSL_Lvar)
HSL_Lstd = np.asarray(HSL_Lstd)
HSL_Lmax = np.asarray(HSL_Lmax)
HSL_Lmin = np.asarray(HSL_Lmin)
HSL_Lmdn = np.asarray(HSL_Lmdn)
HSL_LQ25 = np.asarray(HSL_LQ25)
HSL_LQ75 = np.asarray(HSL_LQ75)
HSL_Lmod = np.asarray(HSL_Lmod)
HSL_Lamp = np.asarray(HSL_Lamp)
HSL_Lsta = np.asarray(HSL_Lsta)
HSL_Lgeo = np.asarray(HSL_Lgeo)
HSL_Lhar = np.asarray(HSL_Lhar)
HSL_Lmode_len = np.asarray(HSL_Lmode_len)
HSV_Hmed = np.asarray(HSV_Hmed)
HSV_Hvar = np.asarray(HSV_Hvar)
HSV_Hstd = np.asarray(HSV_Hstd)
HSV_Hmax = np.asarray(HSV_Hmax)
HSV_Hmin = np.asarray(HSV_Hmin)
HSV_Hmdn = np.asarray(HSV_Hmdn)
HSV_HQ25 = np.asarray(HSV_HQ25)
HSV_HQ75 = np.asarray(HSV_HQ75)
HSV_Hmod = np.asarray(HSV_Hmod)
HSV_Hamp = np.asarray(HSV_Hamp)
HSV_Hsta = np.asarray(HSV_Hsta)
HSV_Hgeo = np.asarray(HSV_Hgeo)
HSV_Hhar = np.asarray(HSV_Hhar)
HSV_Hmode_len = np.asarray(HSV_Hmode_len)
HSV_Smed = np.asarray(HSV_Smed)
HSV_Svar = np.asarray(HSV_Svar)
HSV_Sstd = np.asarray(HSV_Sstd)
HSV_Smax = np.asarray(HSV_Smax)
HSV_Smin = np.asarray(HSV_Smin)
HSV_Smdn = np.asarray(HSV_Smdn)
HSV_SQ25 = np.asarray(HSV_SQ25)
HSV_SQ75 = np.asarray(HSV_SQ75)
HSV_Smod = np.asarray(HSV_Smod)
HSV_Samp = np.asarray(HSV_Samp)
HSV_Ssta = np.asarray(HSV_Ssta)
HSV_Sgeo = np.asarray(HSV_Sgeo)
HSV_Shar = np.asarray(HSV_Shar)
HSV_Smode_len = np.asarray(HSV_Smode_len)
HSV_Vmed = np.asarray(HSV_Vmed)
HSV_Vvar = np.asarray(HSV_Vvar)
HSV_Vstd = np.asarray(HSV_Vstd)
HSV_Vmax = np.asarray(HSV_Vmax)
HSV_Vmin = np.asarray(HSV_Vmin)
HSV_Vmdn = np.asarray(HSV_Vmdn)
HSV_VQ25 = np.asarray(HSV_VQ25)
HSV_VQ75 = np.asarray(HSV_VQ75)
HSV_Vmod = np.asarray(HSV_Vmod)
HSV_Vamp = np.asarray(HSV_Vamp)
HSV_Vsta = np.asarray(HSV_Vsta)
HSV_Vgeo = np.asarray(HSV_Vgeo)
HSV_Vhar = np.asarray(HSV_Vhar)
HSV_Vmode_len = np.asarray(HSV_Vmode_len)
LAB_Lmed = np.asarray(LAB_Lmed)
LAB_Lvar = np.asarray(LAB_Lvar)
LAB_Lstd = np.asarray(LAB_Lstd)
LAB_Lmax = np.asarray(LAB_Lmax)
LAB_Lmin = np.asarray(LAB_Lmin)
LAB_Lmdn = np.asarray(LAB_Lmdn)
LAB_LQ25 = np.asarray(LAB_LQ25)
LAB_LQ75 = np.asarray(LAB_LQ75)
LAB_Lmod = np.asarray(LAB_Lmod)
LAB_Lamp = np.asarray(LAB_Lamp)
LAB_Lsta = np.asarray(LAB_Lsta)
LAB_Lgeo = np.asarray(LAB_Lgeo)
LAB_Lhar = np.asarray(LAB_Lhar)
LAB_Lmode_len = np.asarray(LAB_Lmode_len)
LAB_Amed = np.asarray(LAB_Amed)
LAB_Avar = np.asarray(LAB_Avar)
LAB_Astd = np.asarray(LAB_Astd)
LAB_Amax = np.asarray(LAB_Amax)
LAB_Amin = np.asarray(LAB_Amin)
LAB_Amdn = np.asarray(LAB_Amdn)
LAB_AQ25 = np.asarray(LAB_AQ25)
LAB_AQ75 = np.asarray(LAB_AQ75)
LAB_Amod = np.asarray(LAB_Amod)
LAB_Aamp = np.asarray(LAB_Aamp)
LAB_Asta = np.asarray(LAB_Asta)
LAB_Ageo = np.asarray(LAB_Ageo)
LAB_Ahar = np.asarray(LAB_Ahar)
LAB_Amode_len = np.asarray(LAB_Amode_len)
LAB_Bmed = np.asarray(LAB_Bmed)
LAB_Bvar = np.asarray(LAB_Bvar)
LAB_Bstd = np.asarray(LAB_Bstd)
LAB_Bmax = np.asarray(LAB_Bmax)
LAB_Bmin = np.asarray(LAB_Bmin)
LAB_Bmdn = np.asarray(LAB_Bmdn)
LAB_BQ25 = np.asarray(LAB_BQ25)
LAB_BQ75 = np.asarray(LAB_BQ75)
LAB_Bmod = np.asarray(LAB_Bmod)
LAB_Bamp = np.asarray(LAB_Bamp)
LAB_Bsta = np.asarray(LAB_Bsta)
LAB_Bgeo = np.asarray(LAB_Bgeo)
LAB_Bhar = np.asarray(LAB_Bhar)
LAB_Bmode_len = np.asarray(LAB_Bmode_len)
LUV_Lmed = np.asarray(LUV_Lmed)
LUV_Lvar = np.asarray(LUV_Lvar)
LUV_Lstd = np.asarray(LUV_Lstd)
LUV_Lmax = np.asarray(LUV_Lmax)
LUV_Lmin = np.asarray(LUV_Lmin)
LUV_Lmdn = np.asarray(LUV_Lmdn)
LUV_LQ25 = np.asarray(LUV_LQ25)
LUV_LQ75 = np.asarray(LUV_LQ75)
LUV_Lmod = np.asarray(LUV_Lmod)
LUV_Lamp = np.asarray(LUV_Lamp)
LUV_Lsta = np.asarray(LUV_Lsta)
LUV_Lgeo = np.asarray(LUV_Lgeo)
LUV_Lhar = np.asarray(LUV_Lhar)
LUV_Lmode_len = np.asarray(LUV_Lmode_len)
LUV_Umed = np.asarray(LUV_Umed)
LUV_Uvar = np.asarray(LUV_Uvar)
LUV_Ustd = np.asarray(LUV_Ustd)
LUV_Umax = np.asarray(LUV_Umax)
LUV_Umin = np.asarray(LUV_Umin)
LUV_Umdn = np.asarray(LUV_Umdn)
LUV_UQ25 = np.asarray(LUV_UQ25)
LUV_UQ75 = np.asarray(LUV_UQ75)
LUV_Umod = np.asarray(LUV_Umod)
LUV_Uamp = np.asarray(LUV_Uamp)
LUV_Usta = np.asarray(LUV_Usta)
LUV_Ugeo = np.asarray(LUV_Ugeo)
LUV_Uhar = np.asarray(LUV_Uhar)
LUV_Umode_len = np.asarray(LUV_Umode_len)
LUV_Vmed = np.asarray(LUV_Vmed)
LUV_Vvar = np.asarray(LUV_Vvar)
LUV_Vstd = np.asarray(LUV_Vstd)
LUV_Vmax = np.asarray(LUV_Vmax)
LUV_Vmin = np.asarray(LUV_Vmin)
LUV_Vmdn = np.asarray(LUV_Vmdn)
LUV_VQ25 = np.asarray(LUV_VQ25)
LUV_VQ75 = np.asarray(LUV_VQ75)
LUV_Vmod = np.asarray(LUV_Vmod)
LUV_Vamp = np.asarray(LUV_Vamp)
LUV_Vsta = np.asarray(LUV_Vsta)
LUV_Vgeo = np.asarray(LUV_Vgeo)
LUV_Vhar = np.asarray(LUV_Vhar)
LUV_Vmode_len = np.asarray(LUV_Vmode_len)



img_data['Rmed'] = Rmed
img_data['Gmed'] = Gmed
img_data['Bmed'] = Bmed
img_data['Rvar'] = Rvar
img_data['Gvar'] = Gvar
img_data['Bvar'] = Bvar
img_data['Rstd'] = Rstd
img_data['Gstd'] = Gstd
img_data['Bstd'] = Bstd
img_data['Graymed'] = Graymed
img_data['Grayvar'] = Grayvar
img_data['Graystd'] = Graystd
img_data['raio'] = raio
img_data['area'] = area
img_data['perimetro'] = perimetro
img_data['asymetriX'] = assymx
img_data['asymetriY'] = assymy
img_data['difarea'] = Dif_area
img_data['ampabs'] = Amp_cont_abs
img_data['Rmax'] = Rmax
img_data['Rmin'] = Rmin
img_data['Rmdn'] = Rmdn
img_data['RQ25'] = RQ25  
img_data['RQ75'] = RQ75
img_data['Rmod'] = Rmod
img_data['Ramp'] = Ramp
img_data['Rsta'] = Rsta
img_data['Rgeo'] = Rgeo
img_data['Rhar'] = Rhar
img_data['Gmax'] = Gmax
img_data['Gmin'] = Gmin
img_data['Gmdn'] = Gmdn
img_data['GQ25'] = GQ25
img_data['GQ75'] = GQ75
img_data['Gmod'] = Gmod
img_data['Gamp'] = Gamp
img_data['Gsta'] = Gsta
img_data['Ggeo'] = Ggeo
img_data['Ghar'] = Ghar
img_data['Bmax'] = Bmax
img_data['Bmin'] = Bmin
img_data['Bmdn'] = Bmdn
img_data['BQ25'] = BQ25
img_data['BQ75'] = BQ75
img_data['Bmod'] = Bmod
img_data['Bamp'] = Bamp
img_data['Bsta'] = Bsta
img_data['Bgeo'] = Bgeo
img_data['Bhar'] = Bhar 
img_data['Graymax'] = Graymax
img_data['Graymin'] = Graymin
img_data['Graymdn'] = Graymdn
img_data['GrayQ25'] = GrayQ25
img_data['GrayQ75'] = GrayQ75
img_data['Graymod'] = Graymod
img_data['Grayamp'] = Grayamp
img_data['Graysta'] = Graysta
img_data['Graygeo'] = Graygeo
img_data['Grayhar'] = Grayhar
img_data['Graymax'] = Graymax
img_data['Raioamp'] = Raioamp
img_data['Raiomax'] = Raiomax
img_data['Raiomin'] = Raiomin
img_data['Raiomed'] = Raiomed
img_data['Raiomdn'] = Raiomdn
img_data['RaioQ25'] = RaioQ25
img_data['RaioQ75'] = RaioQ75
img_data['Raiomed'] = Raiomod
img_data['Raiosta'] = Raiosta
img_data['Raiogeo'] = Raiogeo
img_data['Raiohar'] = Raiohar
img_data['AmpSup']  = AmpSup
img_data['AmpInf']  = AmpInf
img_data['Rmode_len']  = Rmode_len
img_data['Gmode_len']  = Gmode_len
img_data['Bmode_len']  = Bmode_len
img_data['Graymode_len']  = Graymode_len
img_data['Rmed_adap'] = Rmed_adap
img_data['Rvar_adap']  = Rvar_adap
img_data['Rstd_adap']  = Rstd_adap
img_data['Rmax_adap']  = Rmax_adap
img_data['Rmin_adap']  = Rmin_adap
img_data['Rmdn_adap']  = Rmdn_adap
img_data['RQ25_adap']  = RQ25_adap
img_data['RQ75_adap'] = RQ75_adap
img_data['Rmod_adap']  = Rmod_adap
img_data['Ramp_adap']  = Ramp_adap
img_data['Rsta_adap']  = Rsta_adap 
img_data['Rgeo_adap']  = Rgeo_adap
img_data['Rhar_adap']  = Rhar_adap
img_data['Rmode_adap_len']  = Rmode_adap_len
img_data['Gmed_adap']  = Gmed_adap
img_data['Gvar_adap']  = Gvar_adap 
img_data['Gstd_adap'] = Gstd_adap
img_data['Gmax_adap']  = Gmax_adap
img_data['Gmin_adap']  = Gmin_adap
img_data['Gmdn_adap']  = Gmdn_adap
img_data['GQ25_adap']  = GQ25_adap
img_data['GQ75_adap']  = GQ75_adap
img_data['Gmod_adap']  = Gmod_adap
img_data['Gamp_adap']  = Gamp_adap
img_data['Gsta_adap']  = Gsta_adap
img_data['Ggeo_adap']  = Ggeo_adap
img_data['Ghar_adap']  = Ghar_adap
img_data['Gmode_adap_len']  = Gmode_adap_len
img_data['Bmed_adap']  = Bmed_adap
img_data['Bvar_adap']  = Bvar_adap
img_data['Bstd_adap']  = Bstd_adap 
img_data['Bmax_adap']  = Bmax_adap 
img_data['Bmin_adap']  = Bmin_adap 
img_data['Bmdn_adap']  = Bmdn_adap 
img_data['BQ25_adap']  = BQ25_adap 
img_data['BQ75_adap']  = BQ75_adap
img_data['Bmod_adap']  = Bmod_adap
img_data['Bamp_adap']  = Bamp_adap
img_data['Bsta_adap']  = Bsta_adap
img_data['Bgeo_adap']  = Bgeo_adap
img_data['Bhar_adap']  = Bhar_adap
img_data['Bmode_adap_len']  = Bmode_adap_len
img_data['Graymed_adap']  = Graymed_adap
img_data['Grayvar_adap']  = Grayvar_adap 
img_data['Graystd_adap']  = Graystd_adap 
img_data['Graymax_adap']  = Graymax_adap
img_data['Graymin_adap']  = Graymin_adap
img_data['Graymdn_adap']  = Graymdn_adap
img_data['GrayQ25_adap']  = GrayQ25_adap
img_data['GrayQ75_adap']  = GrayQ75_adap
img_data['Graymod_adap']  = Graymod_adap
img_data['Grayamp_adap']  = Grayamp_adap
img_data['Graysta_adap']  = Graysta_adap
img_data['Graygeo_adap']  = Graygeo_adap
img_data['Grayhar_adap']  = Grayhar_adap
img_data['Graymode_adap_len']   = Graymode_adap_len
img_data['HSL_Hmed'] = HSL_Hmed
img_data['HSL_Hvar'] = HSL_Hvar
img_data['HSL_Hstd'] = HSL_Hstd
img_data['HSL_Hmax'] = HSL_Hmax
img_data['HSL_Hmin'] = HSL_Hmin
img_data['HSL_Hmdn'] = HSL_Hmdn
img_data['HSL_HQ25'] = HSL_HQ25
img_data['HSL_HQ75'] = HSL_HQ75
img_data['HSL_Hmod'] = HSL_Hmod
img_data['HSL_Hamp'] = HSL_Hamp
img_data['HSL_Hsta'] = HSL_Hsta
img_data['HSL_Hgeo'] = HSL_Hgeo
img_data['HSL_Hhar'] = HSL_Hhar
img_data['HSL_Hmode_len'] = HSL_Hmode_len
img_data['HSL_Smed'] = HSL_Smed
img_data['HSL_Svar'] = HSL_Svar
img_data['HSL_Sstd'] = HSL_Sstd
img_data['HSL_Smax'] = HSL_Smax
img_data['HSL_Smin'] = HSL_Smin
img_data['HSL_Smdn'] = HSL_Smdn
img_data['HSL_SQ25'] = HSL_SQ25
img_data['HSL_SQ75'] = HSL_SQ75
img_data['HSL_Smod'] = HSL_Smod
img_data['HSL_Samp'] = HSL_Samp 
img_data['HSL_Ssta'] = HSL_Ssta
img_data['HSL_Sgeo'] = HSL_Sgeo
img_data['HSL_Shar'] = HSL_Shar
img_data['HSL_Smode_len'] = HSL_Smode_len
img_data['HSL_Lmed'] = HSL_Lmed
img_data['HSL_Lvar'] = HSL_Lvar
img_data['HSL_Lstd'] = HSL_Lstd
img_data['HSL_Lmax'] = HSL_Lmax
img_data['HSL_Lmin'] = HSL_Lmin
img_data['HSL_Lmdn'] = HSL_Lmdn
img_data['HSL_LQ25'] = HSL_LQ25
img_data['HSL_LQ75'] = HSL_LQ75
img_data['HSL_Lmod'] = HSL_Lmod
img_data['HSL_Lamp'] = HSL_Lamp
img_data['HSL_Lsta'] = HSL_Lsta
img_data['HSL_Lgeo'] = HSL_Lgeo
img_data['HSL_Lhar'] = HSL_Lhar
img_data['HSL_Lmode_len'] = HSL_Lmode_len
img_data['HSV_Hmed'] = HSV_Hmed
img_data['HSV_Hvar'] = HSV_Hvar
img_data['HSV_Hstd'] = HSV_Hstd
img_data['HSV_Hmax'] = HSV_Hmax
img_data['HSV_Hmin'] = HSV_Hmin
img_data['HSV_Hmdn'] = HSV_Hmdn
img_data['HSV_HQ25'] = HSV_HQ25
img_data['HSV_HQ75'] = HSV_HQ75
img_data['HSV_Hmod'] = HSV_Hmod
img_data['HSV_Hamp'] = HSV_Hamp
img_data['HSV_Hsta'] = HSV_Hsta
img_data['HSV_Hgeo'] = HSV_Hgeo
img_data['HSV_Hhar'] = HSV_Hhar
img_data['HSV_Hmode_len'] = HSV_Hmode_len
img_data['HSV_Smed'] = HSV_Smed
img_data['HSV_Svar'] = HSV_Svar
img_data['HSV_Sstd'] = HSV_Sstd
img_data['HSV_Smax'] = HSV_Smax
img_data['HSV_Smin'] = HSV_Smin
img_data['HSV_Smdn'] = HSV_Smdn 
img_data['HSV_SQ25'] = HSV_SQ25
img_data['HSV_SQ75'] = HSV_SQ75
img_data['HSV_Smod'] = HSV_Smod
img_data['HSV_Samp'] = HSV_Samp
img_data['HSV_Ssta'] = HSV_Ssta
img_data['HSV_Sgeo'] = HSV_Sgeo
img_data['HSV_Shar'] = HSV_Shar
img_data['HSV_Smode_len'] = HSV_Smode_len
img_data['HSV_Vmed'] = HSV_Vmed
img_data['HSV_Vvar'] = HSV_Vvar
img_data['HSV_Vstd'] = HSV_Vstd
img_data['HSV_Vmax'] = HSV_Vmax
img_data['HSV_Vmin'] = HSV_Vmin 
img_data['HSV_Vmdn'] = HSV_Vmdn
img_data['HSV_VQ25'] = HSV_VQ25
img_data['HSV_VQ75'] = HSV_VQ75
img_data['HSV_Vmod'] = HSV_Vmod
img_data['HSV_Vamp'] = HSV_Vamp
img_data['HSV_Vsta'] = HSV_Vsta
img_data['HSV_Vgeo'] = HSV_Vgeo
img_data['HSV_Vhar'] = HSV_Vhar
img_data['HSV_Vmode_len'] = HSV_Vmode_len
img_data['LAB_Lmed'] = LAB_Lmed
img_data['LAB_Lvar'] = LAB_Lvar
img_data['LAB_Lstd'] = LAB_Lstd
img_data['LAB_Lmax'] = LAB_Lmax
img_data['LAB_Lmin'] = LAB_Lmin
img_data['LAB_Lmdn'] = LAB_Lmdn
img_data['LAB_LQ25'] = LAB_LQ25
img_data['LAB_LQ75'] = LAB_LQ75
img_data['LAB_Lmod'] = LAB_Lmod
img_data['LAB_Lamp'] = LAB_Lamp
img_data['LAB_Lsta'] = LAB_Lsta
img_data['LAB_Lgeo'] = LAB_Lgeo
img_data['LAB_Lhar'] = LAB_Lhar
img_data['LAB_Lmode_len'] = LAB_Lmode_len
img_data['LAB_Amed'] = LAB_Amed
img_data['LAB_Avar'] = LAB_Avar
img_data['LAB_Astd'] = LAB_Astd
img_data['LAB_Amax'] = LAB_Amax
img_data['LAB_Amin'] = LAB_Amin
img_data['LAB_Amdn'] = LAB_Amdn
img_data['LAB_AQ25'] = LAB_AQ25
img_data['LAB_AQ75'] = LAB_AQ25
img_data['LAB_Amod'] = LAB_Amod
img_data['LAB_Aamp'] = LAB_Aamp
img_data['LAB_Asta'] = LAB_Asta
img_data['LAB_Ageo'] = LAB_Ageo
img_data['LAB_Ahar'] = LAB_Ahar
img_data['LAB_Amode_len'] = LAB_Amode_len
img_data['LAB_Bmed'] = LAB_Bmed
img_data['LAB_Bvar'] = LAB_Bvar
img_data['LAB_Bstd'] = LAB_Bstd
img_data['LAB_Bmax'] = LAB_Bmax
img_data['LAB_Bmin'] = LAB_Bmin
img_data['LAB_Bmdn'] = LAB_Bmdn
img_data['LAB_BQ25'] = LAB_BQ25
img_data['LAB_BQ75'] = LAB_BQ75
img_data['LAB_Bmod'] = LAB_Bmod
img_data['LAB_Bamp'] = LAB_Bamp
img_data['LAB_Bsta'] = LAB_Bsta
img_data['LAB_Bgeo'] = LAB_Bgeo
img_data['LAB_Bhar'] = LAB_Bhar
img_data['LAB_Bmode_len'] = LAB_Bmode_len
img_data['LUV_Lmed'] = LUV_Lmed
img_data['LUV_Lvar'] = LUV_Lvar
img_data['LUV_Lstd'] = LUV_Lstd
img_data['LUV_Lmax'] = LUV_Lmax
img_data['LUV_Lmin'] = LUV_Lmin
img_data['LUV_Lmdn'] = LUV_Lmdn
img_data['LUV_LQ25'] = LUV_LQ25
img_data['LUV_LQ75'] = LUV_LQ75
img_data['LUV_Lmod'] = LUV_Lmod
img_data['LUV_Lamp'] = LUV_Lamp
img_data['LUV_Lsta'] = LUV_Lsta
img_data['LUV_Lgeo'] = LUV_Lgeo
img_data['LUV_Lhar'] = LUV_Lhar
img_data['LUV_Lmode_len'] = LUV_Lmode_len
img_data['LUV_Umed'] = LUV_Umed
img_data['LUV_Uvar'] = LUV_Uvar
img_data['LUV_Ustd'] = LUV_Ustd
img_data['LUV_Umax'] = LUV_Umax
img_data['LUV_Umin'] = LUV_Umin
img_data['LUV_Umdn'] = LUV_Umdn
img_data['LUV_UQ25'] = LUV_UQ25
img_data['LUV_UQ75'] = LUV_UQ75
img_data['LUV_Umod'] = LUV_Umod
img_data['LUV_Uamp'] = LUV_Uamp
img_data['LUV_Usta'] = LUV_Usta
img_data['LUV_Ugeo'] = LUV_Ugeo
img_data['LUV_Uhar'] = LUV_Uhar
img_data['LUV_Umode_len'] = LUV_Umode_len
img_data['LUV_Vmed'] = LUV_Vmed
img_data['LUV_Vvar'] = LUV_Vvar
img_data['LUV_Vstd'] = LUV_Vstd
img_data['LUV_Vmax'] = LUV_Vmax
img_data['LUV_Vmin'] = LUV_Vmin
img_data['LUV_Vmdn'] = LUV_Vmdn
img_data['LUV_VQ25'] = LUV_VQ25
img_data['LUV_VQ75'] = LUV_VQ75
img_data['LUV_Vmod'] = LUV_Vmod
img_data['LUV_Vamp'] = LUV_Vamp
img_data['LUV_Vsta'] = LUV_Vsta
img_data['LUV_Vgeo'] = LUV_Vgeo
img_data['LUV_Vhar'] = LUV_Vhar
img_data['LUV_Vmode_len'] = LUV_Vmode_len



In [23]:
img_data.shape

(200, 300)

In [24]:
img_data.head()

,img_id,Rmed,Gmed,Bmed,Rvar,Gvar,Bvar,Rstd,Gstd,Bstd,Graymed,Grayvar,Graystd,raio,area,perimetro,asymetriX,asymetriY,difarea,ampabs,Rmax,Rmin,Rmdn,RQ25,RQ75,Rmod,Ramp,Rsta,Rgeo,Rhar,Gmax,Gmin,Gmdn,GQ25,GQ75,Gmod,Gamp,Gsta,Ggeo,Ghar,...,LUV_Lstd,LUV_Lmax,LUV_Lmin,LUV_Lmdn,LUV_LQ25,LUV_LQ75,LUV_Lmod,LUV_Lamp,LUV_Lsta,LUV_Lgeo,LUV_Lhar,LUV_Lmode_len,LUV_Umed,LUV_Uvar,LUV_Ustd,LUV_Umax,LUV_Umin,LUV_Umdn,LUV_UQ25,LUV_UQ75,LUV_Umod,LUV_Uamp,LUV_Usta,LUV_Ugeo,LUV_Uhar,LUV_Umode_len,LUV_Vmed,LUV_Vvar,LUV_Vstd,LUV_Vmax,LUV_Vmin,LUV_Vmdn,LUV_VQ25,LUV_VQ75,LUV_Vmod,LUV_Vamp,LUV_Vsta,LUV_Vgeo,LUV_Vhar,LUV_Vmode_len
0,IMD003,141.026145,99.907090,95.599948,666.966775,526.681733,475.752436,25.825700,22.949548,21.811750,111.711304,553.897024,23.535017,144,49483.5,909.335130,908,534,11958.897376,0.335784,255,45,138.0,123.0,157.0,128,210,20.383260,138.726852,136.463367,255,2,97.0,85.0,113.0,96,253,17.622184,97.32232,94.482537,...,23.721640,254.0,14.0,115.0,102.0,132.0,113.0,240.0,18.431535,115.244210,112.811707,1,115.783022,9.068789,3.011443,129,93,116.0,114.0,118.0,116,36,2.390265,115.743774,115.704428,1,144.750281,9.690762,3.112999,158,131,145.0,143.0,147.0,143,27,2.520738,144.716917,144.683662,1
1,IMD009,174.015069,116.736129,81.192699,507.485333,485.600350,389.236582,22.527435,22.036342,19.729080,129.811612,472.573475,21.738755,133,49747.0,839.577770,78,896,12413.876730,0.175739,255,45,175.0,156.0,193.0,160,210,19.193162,172.489040,170.867016,255,0,116.0,100.0,135.0,102,255,18.572493,0.00000,0.000000,...,21.446395,253.0,13.0,138.0,121.0,156.0,123.0,240.0,18.106582,136.032494,134.014615,1,127.102048,6.780058,2.603854,139,97,127.0,125.0,129.0,128,42,2.080070,127.075215,127.048209,1,165.826740,17.690416,4.205998,179,140,166.0,163.0,169.0,167,39,3.422543,165.773142,165.719271,1
2,IMD016,170.306554,91.814428,53.234401,603.953999,435.527667,312.169675,24.575476,20.869300,17.668324,110.887407,453.013872,21.284123,181,73324.5,1089.442777,1654,2458,14639.691990,0.312090,255,73,169.0,151.0,188.0,156,182,20.263873,168.534807,166.763142,255,0,89.0,76.0,105.0,83,255,16.683702,0.00000,0.000000,...,21.131143,252.0,30.0,119.0,105.0,135.0,110.0,222.0,17.155740,118.849109,117.016566,1,140.293178,18.438867,4.294050,155,98,141.0,137.0,143.0,142,57,3.449991,140.226800,140.159710,1,167.593579,26.853305,5.182017,184,145,167.0,164.0,171.0,165,39,4.262179,167.513653,167.433913,1
3,IMD022,179.164652,99.801229,38.776840,639.841271,563.731281,352.310635,25.295084,23.743026,18.769940,116.574021,546.427010,23.375778,236,119377.5,1592.971778,719,325,18317.579080,0.380035,255,74,177.0,160.0,198.0,163,181,21.037037,177.380051,175.598128,225,0,97.0,81.0,118.0,82,225,19.843010,0.00000,0.000000,...,22.873491,232.0,31.0,126.0,110.0,146.0,115.0,201.0,19.119042,126.070102,124.029360,1,142.234172,13.494300,3.673459,156,112,142.0,140.0,145.0,143,44,2.921374,142.186613,142.138924,1,176.139313,37.688110,6.139064,195,145,176.0,171.0,181.0,172,50,5.086846,176.032638,175.926276,1
4,IMD024,165.550278,110.724523,76.140293,1322.501257,1041.629865,701.094420,36.366210,32.274291,26.478188,123.175569,1066.198326,32.652692,153,36656.5,792.707712,406,530,10083.023408,0.498939,246,68,169.0,136.0,195.0,184,178,30.897303,161.271661,156.739801,200,28,110.0,84.0,136.0,108,172,27.197246,105.79234,100.704039,...,32.771374,211.0,40.0,132.0,104.0,157.0,129.0,171.0,27.661656,126.398991,121.809163,1,125.670514,16.715673,4.088481,137,111,127.0,123.0,129.0,129,26,3.335670,125.602913,125.534198,1,164.661350,42.476885,6.517429,180,145,166.0,159.0,170.0,168,35,5.523483,164.530698,164.398398,1


In [25]:
full_img_data = pd.concat([img_data, result], axis=1, join='inner')
full_img_data

,img_id,Rmed,Gmed,Bmed,Rvar,Gvar,Bvar,Rstd,Gstd,Bstd,Graymed,Grayvar,Graystd,raio,area,perimetro,asymetriX,asymetriY,difarea,ampabs,Rmax,Rmin,Rmdn,RQ25,RQ75,Rmod,Ramp,Rsta,Rgeo,Rhar,Gmax,Gmin,Gmdn,GQ25,GQ75,Gmod,Gamp,Gsta,Ggeo,Ghar,...,LUV_LQ75,LUV_Lmod,LUV_Lamp,LUV_Lsta,LUV_Lgeo,LUV_Lhar,LUV_Lmode_len,LUV_Umed,LUV_Uvar,LUV_Ustd,LUV_Umax,LUV_Umin,LUV_Umdn,LUV_UQ25,LUV_UQ75,LUV_Umod,LUV_Uamp,LUV_Usta,LUV_Ugeo,LUV_Uhar,LUV_Umode_len,LUV_Vmed,LUV_Vvar,LUV_Vstd,LUV_Vmax,LUV_Vmin,LUV_Vmdn,LUV_VQ25,LUV_VQ75,LUV_Vmod,LUV_Vamp,LUV_Vsta,LUV_Vgeo,LUV_Vhar,LUV_Vmode_len,Class,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma
0,IMD003,141.026145,99.907090,95.599948,666.966775,526.681733,475.752436,25.825700,22.949548,21.811750,111.711304,553.897024,23.535017,144,49483.5,909.335130,908,534,11958.897376,0.335784,255,45,138.0,123.0,157.0,128,210,20.383260,138.726852,136.463367,255,2,97.0,85.0,113.0,96,253,17.622184,97.322320,94.482537,...,132.0,113.0,240.0,18.431535,115.244210,112.811707,1,115.783022,9.068789,3.011443,129,93,116.0,114.0,118.0,116,36,2.390265,115.743774,115.704428,1,144.750281,9.690762,3.112999,158,131,145.0,143.0,147.0,143,27,2.520738,144.716917,144.683662,1,Common Nevus,0,1,0,0
1,IMD009,174.015069,116.736129,81.192699,507.485333,485.600350,389.236582,22.527435,22.036342,19.729080,129.811612,472.573475,21.738755,133,49747.0,839.577770,78,896,12413.876730,0.175739,255,45,175.0,156.0,193.0,160,210,19.193162,172.489040,170.867016,255,0,116.0,100.0,135.0,102,255,18.572493,0.000000,0.000000,...,156.0,123.0,240.0,18.106582,136.032494,134.014615,1,127.102048,6.780058,2.603854,139,97,127.0,125.0,129.0,128,42,2.080070,127.075215,127.048209,1,165.826740,17.690416,4.205998,179,140,166.0,163.0,169.0,167,39,3.422543,165.773142,165.719271,1,Common Nevus,0,1,0,0
2,IMD016,170.306554,91.814428,53.234401,603.953999,435.527667,312.169675,24.575476,20.869300,17.668324,110.887407,453.013872,21.284123,181,73324.5,1089.442777,1654,2458,14639.691990,0.312090,255,73,169.0,151.0,188.0,156,182,20.263873,168.534807,166.763142,255,0,89.0,76.0,105.0,83,255,16.683702,0.000000,0.000000,...,135.0,110.0,222.0,17.155740,118.849109,117.016566,1,140.293178,18.438867,4.294050,155,98,141.0,137.0,143.0,142,57,3.449991,140.226800,140.159710,1,167.593579,26.853305,5.182017,184,145,167.0,164.0,171.0,165,39,4.262179,167.513653,167.433913,1,Common Nevus,0,1,0,0
3,IMD022,179.164652,99.801229,38.776840,639.841271,563.731281,352.310635,25.295084,23.743026,18.769940,116.574021,546.427010,23.375778,236,119377.5,1592.971778,719,325,18317.579080,0.380035,255,74,177.0,160.0,198.0,163,181,21.037037,177.380051,175.598128,225,0,97.0,81.0,118.0,82,225,19.843010,0.000000,0.000000,...,146.0,115.0,201.0,19.119042,126.070102,124.029360,1,142.234172,13.494300,3.673459,156,112,142.0,140.0,145.0,143,44,2.921374,142.186613,142.138924,1,176.139313,37.688110,6.139064,195,145,176.0,171.0,181.0,172,50,5.086846,176.032638,175.926276,1,Common Nevus,0,1,0,0
4,IMD024,165.550278,110.724523,76.140293,1322.501257,1041.629865,701.094420,36.366210,32.274291,26.478188,123.175569,1066.198326,32.652692,153,36656.5,792.707712,406,530,10083.023408,0.498939,246,68,169.0,136.0,195.0,184,178,30.897303,161.271661,156.739801,200,28,110.0,84.0,136.0,108,172,27.197246,105.792340,100.704039,...,157.0,129.0,171.0,27.661656,126.398991,121.809163,1,125.670514,16.715673,4.088481,137,111,127.0,123.0,129.0,129,26,3.335670,125.602913,125.534198,1,164.661350,42.476885,6.517429,180,145,166.0,159.0,170.0,168,35,5.523483,164.530698,164.398398,1,Common Nevus,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IMD424,199.091937,150.993300,149.435851,2689.032137,1804.439557,1473.575081,51.855879,42.478695,38.387173,165.207616,1891.551483,43.4

In [26]:
full_all_data = pd.concat([doc_data,img_data.drop(['img_id'],axis=1), result], axis=1, join='inner')
full_all_data

,img_id,asymmetry,pigment_net,dots_glob,streaks,regressionA,blue_whit,Color_W,Color_R,Color_L_B,Color_D_B,Color_B_G,Color_B,Rmed,Gmed,Bmed,Rvar,Gvar,Bvar,Rstd,Gstd,Bstd,Graymed,Grayvar,Graystd,raio,area,perimetro,asymetriX,asymetriY,difarea,ampabs,Rmax,Rmin,Rmdn,RQ25,RQ75,Rmod,Ramp,Rsta,...,LUV_LQ75,LUV_Lmod,LUV_Lamp,LUV_Lsta,LUV_Lgeo,LUV_Lhar,LUV_Lmode_len,LUV_Umed,LUV_Uvar,LUV_Ustd,LUV_Umax,LUV_Umin,LUV_Umdn,LUV_UQ25,LUV_UQ75,LUV_Umod,LUV_Uamp,LUV_Usta,LUV_Ugeo,LUV_Uhar,LUV_Umode_len,LUV_Vmed,LUV_Vvar,LUV_Vstd,LUV_Vmax,LUV_Vmin,LUV_Vmdn,LUV_VQ25,LUV_VQ75,LUV_Vmod,LUV_Vamp,LUV_Vsta,LUV_Vgeo,LUV_Vhar,LUV_Vmode_len,Class,histo_diagnosis,Common_Nevus,Atypical_Nevus,Melanoma
0,IMD003,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,0,1,0,0,141.026145,99.907090,95.599948,666.966775,526.681733,475.752436,25.825700,22.949548,21.811750,111.711304,553.897024,23.535017,144,49483.5,909.335130,908,534,11958.897376,0.335784,255,45,138.0,123.0,157.0,128,210,20.383260,...,132.0,113.0,240.0,18.431535,115.244210,112.811707,1,115.783022,9.068789,3.011443,129,93,116.0,114.0,118.0,116,36,2.390265,115.743774,115.704428,1,144.750281,9.690762,3.112999,158,131,145.0,143.0,147.0,143,27,2.520738,144.716917,144.683662,1,Common Nevus,0,1,0,0
1,IMD009,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0,174.015069,116.736129,81.192699,507.485333,485.600350,389.236582,22.527435,22.036342,19.729080,129.811612,472.573475,21.738755,133,49747.0,839.577770,78,896,12413.876730,0.175739,255,45,175.0,156.0,193.0,160,210,19.193162,...,156.0,123.0,240.0,18.106582,136.032494,134.014615,1,127.102048,6.780058,2.603854,139,97,127.0,125.0,129.0,128,42,2.080070,127.075215,127.048209,1,165.826740,17.690416,4.205998,179,140,166.0,163.0,169.0,167,39,3.422543,165.773142,165.719271,1,Common Nevus,0,1,0,0
2,IMD016,Simétrico,Típico,Típico,Ausente,Ausente,Ausente,0,0,1,1,0,0,170.306554,91.814428,53.234401,603.953999,435.527667,312.169675,24.575476,20.869300,17.668324,110.887407,453.013872,21.284123,181,73324.5,1089.442777,1654,2458,14639.691990,0.312090,255,73,169.0,151.0,188.0,156,182,20.263873,...,135.0,110.0,222.0,17.155740,118.849109,117.016566,1,140.293178,18.438867,4.294050,155,98,141.0,137.0,143.0,142,57,3.449991,140.226800,140.159710,1,167.593579,26.853305,5.182017,184,145,167.0,164.0,171.0,165,39,4.262179,167.513653,167.433913,1,Common Nevus,0,1,0,0
3,IMD022,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,0,0,0,179.164652,99.801229,38.776840,639.841271,563.731281,352.310635,25.295084,23.743026,18.769940,116.574021,546.427010,23.375778,236,119377.5,1592.971778,719,325,18317.579080,0.380035,255,74,177.0,160.0,198.0,163,181,21.037037,...,146.0,115.0,201.0,19.119042,126.070102,124.029360,1,142.234172,13.494300,3.673459,156,112,142.0,140.0,145.0,143,44,2.921374,142.186613,142.138924,1,176.139313,37.688110,6.139064,195,145,176.0,171.0,181.0,172,50,5.086846,176.032638,175.926276,1,Common Nevus,0,1,0,0
4,IMD024,Simétrico,Típico,Ausente,Ausente,Ausente,Ausente,0,0,1,1,0,0,165.550278,110.724523,76.140293,1322.501257,1041.629865,701.094420,36.366210,32.274291,26.478188,123.175569,1066.198326,32.652692,153,36656.5,792.707712,406,530,10083.023408,0.498939,246,68,169.0,136.0,195.0,184,178,30.897303,...,157.0,129.0,171.0,27.661656,126.398991,121.809163,1,125.670514,16.715673,4.088481,137,111,127.0,123.0,129.0,129,26,3.335670,125.602913,125.534198,1,164.661350,42.476885,6.517429,180,145,166.0,159.0,170.0,168,35,5.523483,164.530698,164.398398,1,Common Nevus,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,IMD424,Totalmente A.,Atípico,Atípico,Presente,Ausente,Presente,0,0,1,1,1,1,199.091937,150.993300,149.435851,2689.032137,1804.439557,1473.575081,51.855879,42.478695,38.387173,165.20

In [27]:
full_img_data.to_csv("Dados_Imagem_PH2_Final.csv")
full_doc_data.to_csv('Dados_Doc_PH2_Final.csv')
full_all_data.to_csv('Full_Data_PH2_Final.csv')

In [28]:
full_all_data.describe

<bound method NDFrame.describe of      img_id      asymmetry pigment_net  ... Common_Nevus Atypical_Nevus Melanoma
0    IMD003      Simétrico      Típico  ...            1              0        0
1    IMD009      Simétrico      Típico  ...            1              0        0
2    IMD016      Simétrico      Típico  ...            1              0        0
3    IMD022      Simétrico      Típico  ...            1              0        0
4    IMD024      Simétrico      Típico  ...            1              0        0
..      ...            ...         ...  ...          ...            ...      ...
195  IMD424  Totalmente A.     Atípico  ...            0              0        1
196  IMD425  Totalmente A.     Atípico  ...            0              0        1
197  IMD426  Totalmente A.     Atípico  ...            0              0        1
198  IMD429      Simétrico     Atípico  ...            0              0        1
199  IMD435  Totalmente A.     Atípico  ...            0              0    

In [29]:
final_time = time.time() - start_time
print('O código levou {}s para finalizar'.format(final_time))

O código levou 2470.543568611145s para finalizar


In [30]:
final_time/60

41.175726143519086